In [ ]:
!pip install tensorflow==2.18.0
!pip install gtts
!pip install Keras-Preprocessing
!pip install tensorflow_addons
!pip install --quiet vit-keras
!pip install pycocoevalcap
!pip install numpy==2.0.0
!pip install pandas==2.2.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 615.5/615.5 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 73.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 117.5 MB/s eta 0:00:00
  Attempting uninstall: ml-dtypes
    Found existing installation: ml_dtypes 0.5.3
    Uninstalling ml_dtypes-0.5.3:
      Successfully uninstalled ml_dtypes-0.5.3
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.19.0
    Uninstalling tensorboard-2.19.0:
      Successfully uninstalled tensorboard-2.19.0
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.19.0
    Uninstalling tensorflow-2.19.0:
      Successfully uninstalled tensorflow-2.19.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tf-keras 2.19.0 requires tensorflow<2.20,>=2.19, but you have tensorflow 2.18.0 

In [ ]:
import tensorflow as tf
from collections import Counter
import keras
import string
import random
import numpy as np
import pandas as pd
from numpy import array
from PIL import Image
import pickle
import matplotlib.pyplot as plt
import keras
import sys, time, os, warnings
warnings.filterwarnings("ignore")
import re
from tqdm import tqdm
from nltk.translate.bleu_score import sentence_bleu
from keras_preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense, BatchNormalization
from keras.layers import LSTM
from keras.layers import Embedding
from keras.layers import Dropout
from keras.layers import add
from keras.callbacks import ModelCheckpoint
from keras_preprocessing.image import load_img, img_to_array
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
# import keras
# from keras_cv.layers import DropPath
from tensorflow.keras import ops
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
# import tensorflow.keras.layers.Layer

  # only for dataloader
import tensorflow_datasets as tfds  # for flower dataset

from skimage.data import chelsea
import matplotlib.pyplot as plt
import numpy as np
# from vit_keras import vit

In [ ]:
import os
import shutil

# Path to the directory containing all the images
image_directory = 'Image directory path'

# Path to the file containing all the captions
caption_file = 'Caption file path'

# Path to the text file containing the training image IDs
training_ids_file = 'Training image ids'

# Create the output folder if it doesn't exist
output_folder = 'all_data'
os.makedirs(output_folder, exist_ok=True)

# Path to the output image folder
output_image_folder = os.path.join(output_folder, 'training_images')
os.makedirs(output_image_folder, exist_ok=True)

# Path to the output text file
output_text_file = os.path.join(output_folder, 'training_captions.txt')

# Read the training image IDs from the text file
with open(training_ids_file, 'r') as f:
    training_ids = f.read().splitlines()


TRAIN_LIMIT = "training data size"

training_ids = training_ids[:TRAIN_LIMIT] #for limit of train ids


# Create a dictionary to store the captions for training images
captions = {}

# Read the captions from the caption file
with open(caption_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            image_id, caption = line.split(',', 1)
            image_id = image_id.split('.')[0]  # Remove the file extension for comparison
            if image_id in training_ids:
                if image_id not in captions:
                    captions[image_id] = []
                captions[image_id].append(caption.strip())

# Write the image IDs and captions to the output text file
with open(output_text_file, 'w') as f:
    for image_id in training_ids:
        # Form the image filename
        image_filename = f'{image_id}.jpg'

        # Copy the image to the output image folder
        src_path = os.path.join(image_directory, image_filename)
        dst_path = os.path.join(output_image_folder, image_filename)
        if os.path.exists(src_path):
            shutil.copyfile(src_path, dst_path)
        else:
            print(f"Warning: {image_filename} does not exist in {image_directory}")

        # Write the image ID and captions to the output text file
        if image_id in captions:
            caption_list = captions[image_id]
            for caption in caption_list:
                f.write(f"{image_filename},{caption}\n")
        else:
            f.write(f"{image_filename},\n")


In [ ]:

# Path to the directory containing all the images
image_directory = 'Image directory path'

# Path to the file containing all the captions
caption_file = 'Caption file path'

# Path to the text file containing the training image IDs
val_ids_file = 'Validation image ids'

# Create the output folder if it doesn't exist
output_folder = 'all_data'
os.makedirs(output_folder, exist_ok=True)

# Path to the output image folder
output_image_folder = os.path.join(output_folder, 'validation_images')
os.makedirs(output_image_folder, exist_ok=True)

# Path to the output text file
output_text_file = os.path.join(output_folder, 'validation_captions.txt')

# Read the training image IDs from the text file
with open(val_ids_file, 'r') as f:
    val_ids = f.read().splitlines()

VALID_LIMIT = 'Validation data size'

val_ids = val_ids[:VALID_LIMIT] #for limit of validation ids


# Create a dictionary to store the captions for training images
captions = {}

# Read the captions from the caption file
with open(caption_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            image_id, caption = line.split(',', 1)
            image_id = image_id.split('.')[0]  # Remove the file extension for comparison
            if image_id in training_ids:
                if image_id not in captions:
                    captions[image_id] = []
                captions[image_id].append(caption.strip())

# Write the image IDs and captions to the output text file
with open(output_text_file, 'w') as f:
    for image_id in training_ids:
        # Form the image filename
        image_filename = f'{image_id}.jpg'

        # Copy the image to the output image folder
        src_path = os.path.join(image_directory, image_filename)
        dst_path = os.path.join(output_image_folder, image_filename)
        if os.path.exists(src_path):
            shutil.copyfile(src_path, dst_path)
        else:
            print(f"Warning: {image_filename} does not exist in {image_directory}")

        # Write the image ID and captions to the output text file
        if image_id in captions:
            caption_list = captions[image_id]
            for caption in caption_list:
                f.write(f"{image_filename},{caption}\n")
        else:
            f.write(f"{image_filename},\n")


In [ ]:

# Path to the directory containing all the images
image_directory = 'Image directory path'

# Path to the file containing all the captions
caption_file = 'Caption file path'

# Path to the text file containing the training image IDs
test_ids_file = 'Test image ids'

# Create the output folder if it doesn't exist
output_folder = 'all_data'
os.makedirs(output_folder, exist_ok=True)

# Path to the output image folder
output_image_folder = os.path.join(output_folder, 'testing_images')
os.makedirs(output_image_folder, exist_ok=True)

# Path to the output text file
output_text_file = os.path.join(output_folder, 'testing_captions.txt')

# Read the training image IDs from the text file
with open(test_ids_file, 'r') as f:
    test_ids = f.read().splitlines()

TEST_LIMIT=1000

test_ids = test_ids[:TEST_LIMIT] #for limit of Test ids


# Create a dictionary to store the captions for training images
captions = {}

# Read the captions from the caption file
with open(caption_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            image_id, caption = line.split(',', 1)
            image_id = image_id.split('.')[0]  # Remove the file extension for comparison
            if image_id in training_ids:
                if image_id not in captions:
                    captions[image_id] = []
                captions[image_id].append(caption.strip())

# Write the image IDs and captions to the output text file
with open(output_text_file, 'w') as f:
    for image_id in training_ids:
        # Form the image filename
        image_filename = f'{image_id}.jpg'

        # Copy the image to the output image folder
        src_path = os.path.join(image_directory, image_filename)
        dst_path = os.path.join(output_image_folder, image_filename)
        if os.path.exists(src_path):
            shutil.copyfile(src_path, dst_path)
        else:
            print(f"Warning: {image_filename} does not exist in {image_directory}")

        # Write the image ID and captions to the output text file
        if image_id in captions:
            caption_list = captions[image_id]
            for caption in caption_list:
                f.write(f"{image_filename},{caption}\n")
        else:
            f.write(f"{image_filename},\n")


In [ ]:
# Paths
img_patht = '/content/all_data/training_images/'
caption_file = '/content/all_data/training_captions.txt'

# Initialize lists
all_idst = []
all_img_vectort = []
annotationst = []

# Read the captions from the caption file
with open(caption_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            image_idt, captiont = line.split(',', 1)
            full_flickr_image_patht = os.path.join(img_patht, image_idt)

            all_idst.append(image_idt)
            all_img_vectort.append(full_flickr_image_patht)
            annotationst.append(captiont)

# Shuffling the captions and image names together
img_idst, all_img_vectort, annotationst = shuffle(all_idst, all_img_vectort, annotationst)

num_examples = len(all_idst)

img_idst = img_idst[:num_examples]
all_img_vectort = all_img_vectort[:num_examples]
annotationst = annotationst[:num_examples]

# Creating a DataFrame
dftrain = pd.DataFrame(list(zip(img_idst, all_img_vectort, annotationst)), columns=['ID', 'Path', 'Caption'])
dftrain


In [ ]:
# Paths
img_pathv = '/content/all_data/validation_images/'
caption_file = '/content/all_data/validation_captions.txt'

# Initialize lists
all_idsv = []
all_img_vectorv = []
annotationsv = []

# Read the captions from the caption file
with open(caption_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line:
            image_idv, captionv = line.split(',', 1)
            full_flickr_image_pathv = os.path.join(img_pathv, image_idv)

            all_idsv.append(image_idv)
            all_img_vectorv.append(full_flickr_image_pathv)
            annotationsv.append(captionv)

# Shuffling the captions and image names together
img_idsv, all_img_vectorv, annotationsv = shuffle(all_idsv, all_img_vectorv, annotationsv)

num_examples = len(all_idsv)

img_idsv = img_idsv[:num_examples]
all_img_vectorv = all_img_vectorv[:num_examples]
annotationsv = annotationsv[:num_examples]

# Creating a DataFrame
dfval = pd.DataFrame(list(zip(img_idsv, all_img_vectorv, annotationsv)), columns=['ID', 'Path', 'Caption'])
dfval


In [ ]:
#check total captions and images present in dataset
annotationstotal=annotationst+annotationsv
print("Total captions present in the dataset: "+ str(len(annotationstotal)))
# print("Total images present in the dataset: " + str(len(all_imgstotal)))

#Create vocabulary & counter for the captions
vocabulary = [word.lower() for line in annotationstotal for word in line.split()]
val_count = Counter(vocabulary)
print(val_count)

In [ ]:
img_ids=img_idst+img_idsv
all_img_vector=all_img_vectort+all_img_vectorv
annotations=annotationst+annotationsv
df = pd.DataFrame(list(zip(img_ids,all_img_vector,annotations)),columns =['ID','Path', 'Caption'])
#print(df)
df



In [ ]:
#Visualise the top 20 occuring words in the captions
print('No of total words :',len(val_count))
for word, count in val_count.most_common(20):
  print(word, ": ", count)
lst = val_count.most_common(20)
most_common_words_df = pd.DataFrame(lst, columns = ['Word', 'Count'])
most_common_words_df.plot.bar(x='Word', y='Count', width=0.6, color='skyblue', figsize=(15, 10))
plt.title("Top 20 most frequent words before processing", fontsize = 18, color= 'navy')
plt.xlabel("Words", fontsize = 14, color= 'navy')
plt.ylabel("Count", fontsize = 14, color= 'navy')


In [ ]:
def caption_with_img_plot(image_id, frame) :
  capt = ("\n" *2).join(frame[frame['ID'] == image_id].Caption.to_list())
  fig, ax = plt.subplots()
  ax.set_axis_off()
  idx = pd.DataFrame.ID.to_list().index(image_id)
  im =  Image.open(pd.DataFrame.Path.iloc[idx])
  w, h = im.size[0], im.size[-1]
  ax.imshow(im)
  ax.text(w+50, h, capt, fontsize = 10, color = 'navy')

In [ ]:
#data cleaning
rem_punct = str.maketrans('', '', string.punctuation)
for r in range(len(annotationstotal)) :
  line = annotationstotal[r]
  line = line.split()

  # remove ambiguity
  for word in line:
    if (word!=word.lower()):
      line = [word.lower() for word in line]

  # remove punctuation from each caption and hanging letters
  line = [word.translate(rem_punct) for word in line]
  line = [word for word in line if len(word) > 0]

  # remove numeric values
  line = [word for word in line if word.isalpha()]

  annotationstotal[r] = ' '.join(line)

#add the <start> & <end> token to all those captions as well
annotations = ['<start>' + ' ' + line + ' ' + '<end>' for line in annotationstotal]

#Create a list which contains all the path to the images
all_img_path = all_img_vector
##list contatining captions for an image
annotations[0:5]


In [ ]:
import math
res=0
word_counts ={}
threshold = 2
for line in annotationstotal:
  for word in line.split():
    word_counts[word] = word_counts.get(word, 0) + 1

print(word_counts)

for i in range(len(annotationstotal)):
  #print(type(len(annotations[i].split())))
  res = res + len(annotationstotal[i].split())
res=res-len(annotationstotal)*2
#print ("The number of words in string are : " + str(res))
vocabulary = [word for line in annotationstotal for word in line.split() if word_counts[word] >= threshold]
val_count = Counter(vocabulary)
#unq = val_count
print("No of total unique words in vocabulary after data cleaning excluding <start> and <end>:",val_count)

In [ ]:
# Creating tokenizer
top_word_cnt = val_count
tokenizer = Tokenizer(num_words = top_word_cnt+1, filters= '!"#$%^&*()_+.,:;-?/~`{}[]|\=@ ',
                      lower = True, char_level = False,
                      oov_token = 'UNK')

# Creating word-to-index and index-to-word mappings.
tokenizer.fit_on_texts(annotations)
#transform each text into a sequence of integers
train_seqs = tokenizer.texts_to_sequences(annotations)

# Add PAD token for zero
tokenizer.word_index['PAD'] = 0
tokenizer.index_word[0] = 'PAD'
print(tokenizer.oov_token)
print(tokenizer.index_word[0])

In [ ]:
# Creating a word count for our tokenizer to visualize the Top 20 occuring words after text processing

tokenizer_top_words = [word for line in annotationstotal for word in line.split() ]

#tokenizer_top_words_count
tokenizer_top_words_count = Counter(tokenizer_top_words)

for word, count in tokenizer_top_words_count.most_common(20) :
  print(word, ": ", count)

tokens = tokenizer_top_words_count.most_common(20)
most_com_words_df = pd.DataFrame(tokens, columns = ['Word', 'Count'])

In [ ]:
#plot 20 most common occuring words after processing

lst2=tokenizer_top_words_count.most_common(20)
most_common_words_df=pd.DataFrame(lst2, columns = ['Word','Count'])
most_common_words_df.plot.bar(x = 'Word', y= 'Count', width=0.6, color = 'skyblue', figsize = (15, 10))
plt.title('Top 20 most frequent words after processing', fontsize =18, color= 'navy')
plt.xlabel('Words', fontsize =14, color= 'navy')
plt.ylabel('Counts', fontsize =14, color= 'navy')

In [ ]:
# Pad each vector to the max_length of the captions  store it to a vairable
train_seqs_len = [len(seq) for seq in train_seqs]
longest_sentence_length = max(train_seqs_len)
print(longest_sentence_length)
cap_vector= tf.keras.preprocessing.sequence.pad_sequences(train_seqs, padding= 'post', maxlen = longest_sentence_length,
                                                          dtype='int32', value=0)
print("The shape of Caption vector is :" + str(cap_vector.shape))

# creating list to store preprocessed images and setting up the Image shape
preprocessed_image = []
IMAGE_SHAPE = (256, 256)
#checking image format
#tf.keras.backend.image_data_format()

In [ ]:
class SqueezeAndExcitation(layers.Layer):
    """Squeeze and excitation block.

    Args:
        output_dim: output features dimension, if `None` use same dim as input.
        expansion: expansion ratio.
    """

    def __init__(self, output_dim=None, expansion=0.25, **kwargs):
        super().__init__(**kwargs)
        self.expansion = expansion
        self.output_dim = output_dim

    def build(self, input_shape):
        inp = input_shape[-1]
        self.output_dim = self.output_dim or inp
        self.avg_pool = layers.GlobalAvgPool2D(keepdims=True, name="avg_pool")
        self.fc = [
            layers.Dense(int(inp * self.expansion), use_bias=False, name="fc_0"),
            layers.Activation("gelu", name="fc_1"),
            layers.Dense(self.output_dim, use_bias=False, name="fc_2"),
            layers.Activation("sigmoid", name="fc_3"),
        ]
        super().build(input_shape)

    def call(self, inputs, **kwargs):
        x = self.avg_pool(inputs)
        for layer in self.fc:
            x = layer(x)
        return x * inputs


class ReduceSize(layers.Layer):
    """Down-sampling block.

    Args:
        keepdims: if False spatial dim is reduced and channel dim is increased
    """

    def __init__(self, keepdims=False, **kwargs):
        super().__init__(**kwargs)
        self.keepdims = keepdims

    def build(self, input_shape):
        embed_dim = input_shape[-1]
        dim_out = embed_dim if self.keepdims else 2 * embed_dim
        self.pad1 = layers.ZeroPadding2D(1, name="pad1")
        self.pad2 = layers.ZeroPadding2D(1, name="pad2")
        self.conv = [
            layers.DepthwiseConv2D(
                kernel_size=3, strides=1, padding="valid", use_bias=False, name="conv_0"
            ),
            layers.Activation("gelu", name="conv_1"),
            SqueezeAndExcitation(name="conv_2"),
            layers.Conv2D(
                embed_dim,
                kernel_size=1,
                strides=1,
                padding="valid",
                use_bias=False,
                name="conv_3",
            ),
        ]
        self.reduction = layers.Conv2D(
            dim_out,
            kernel_size=3,
            strides=2,
            padding="valid",
            use_bias=False,
            name="reduction",
        )
        self.norm1 = layers.LayerNormalization(
            -1, 1e-05, name="norm1"
        )  # eps like PyTorch
        self.norm2 = layers.LayerNormalization(-1, 1e-05, name="norm2")

    def call(self, inputs, **kwargs):
        x = self.norm1(inputs)
        xr = self.pad1(x)
        for layer in self.conv:
            xr = layer(xr)
        x = x + xr
        x = self.pad2(x)
        x = self.reduction(x)
        x = self.norm2(x)
        return x


class MLP(layers.Layer):
    """Multi-Layer Perceptron (MLP) block.

    Args:
        hidden_features: hidden features dimension.
        out_features: output features dimension.
        activation: activation function.
        dropout: dropout rate.
    """

    def __init__(
        self,
        hidden_features=None,
        out_features=None,
        activation="gelu",
        dropout=0.0,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.hidden_features = hidden_features
        self.out_features = out_features
        self.activation = activation
        self.dropout = dropout

    def build(self, input_shape):
        self.in_features = input_shape[-1]
        self.hidden_features = self.hidden_features or self.in_features
        self.out_features = self.out_features or self.in_features
        self.fc1 = layers.Dense(self.hidden_features, name="fc1")
        self.act = layers.Activation(self.activation, name="act")
        self.fc2 = layers.Dense(self.out_features, name="fc2")
        self.drop1 = layers.Dropout(self.dropout, name="drop1")
        self.drop2 = layers.Dropout(self.dropout, name="drop2")

    def call(self, inputs, **kwargs):
        x = self.fc1(inputs)
        x = self.act(x)
        x = self.drop1(x)
        x = self.fc2(x)
        x = self.drop2(x)
        return x

In [ ]:
class PatchEmbed(layers.Layer):
    """Patch embedding block.

    Args:
        embed_dim: feature size dimension.
    """

    def __init__(self, embed_dim, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim

    def build(self, input_shape):
        self.pad = layers.ZeroPadding2D(1, name="pad")
        self.proj = layers.Conv2D(self.embed_dim, 3, 2, name="proj")
        self.conv_down = ReduceSize(keepdims=True, name="conv_down")

    def call(self, inputs, **kwargs):
        x = self.pad(inputs)
        x = self.proj(x)
        x = self.conv_down(x)
        return x

In [ ]:
class ConvNextBlock(layers.Layer):
    """ConvNeXt Block as defined in your implementation"""
    def __init__(self, embed_dim, drop_path=0.0, layer_scale_init=1e-6, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.drop_path = drop_path
        self.layer_scale_init = layer_scale_init

    def build(self, input_shape):
        self.dw_conv = layers.DepthwiseConv2D(kernel_size=7, padding="same")
        self.norm = layers.LayerNormalization(epsilon=1e-6)
        self.pw_conv1 = layers.Conv2D(4*self.embed_dim, kernel_size=1)
        self.act = layers.Activation("gelu")
        self.pw_conv2 = layers.Conv2D(self.embed_dim, kernel_size=1)
        self.gamma = self.add_weight(
            shape=(self.embed_dim, ),
            initializer="zeros",
            trainable=True
        )
        self.drop_path_layer = layers.Dropout(rate=self.drop_path) if self.drop_path > 0. else layers.Activation("linear")

    def call(self, inputs, training=False, **kwargs):
        x = inputs
        residual = x
        x = self.dw_conv(x)
        x = self.norm(x)
        x = self.pw_conv1(x)
        x = self.act(x)
        x = self.pw_conv2(x)
        x = self.gamma * x
        x = self.drop_path_layer(x, training=training)
        return residual + x

In [ ]:
class ConvNeXtFeatureStage(layers.Layer):
    """Single ConvNeXt-based feature extraction stage.

    This replaces the original FeatureExtraction layer while maintaining
    the same interface and behavior.

    Args:
        keepdims: bool argument for maintaining the resolution.
    """

    def __init__(self, keepdims=False, **kwargs):
        super().__init__(**kwargs)
        self.keepdims = keepdims

    def build(self, input_shape):
        embed_dim = input_shape[-1]

        # ConvNeXt block for feature extraction
        self.convnext_block = ConvNextBlock(
            embed_dim=embed_dim,
            drop_path=0.0,
            name="convnext_block"
        )

        # Downsampling if not keeping dimensions
        if not self.keepdims:
            self.pad = layers.ZeroPadding2D(1, name="pad")
            self.pool = layers.MaxPool2D(3, 2, name="pool")

        super().build(input_shape)

    def call(self, inputs, **kwargs):
        x = inputs

        # Apply ConvNeXt block for feature extraction
        x = self.convnext_block(x)

        # Apply downsampling if needed
        if not self.keepdims:
            x = self.pad(x)
            x = self.pool(x)

        return x


class GlobalQueryGenerator(layers.Layer):
    """ConvNeXt-based Global query generator that replaces the original.

    This maintains the same interface as the original GlobalQueryGenerator
    but uses ConvNeXt blocks for better feature extraction.

    Args:
        keepdims: list of booleans or single boolean for keeping dimensions
    """

    def __init__(self, keepdims=False, **kwargs):
        super().__init__(**kwargs)
        self.keepdims = keepdims

    def build(self, input_shape):
        # Handle keepdims same as original - convert to list if needed
        if isinstance(self.keepdims, (list, tuple)):
            keepdims_list = list(self.keepdims)
        else:
            # If single boolean, create a reasonable default sequence
            # Based on typical usage: downsample -> downsample -> keep final resolution
            keepdims_list = [False, False, True] if not self.keepdims else [True, True, True]

        self.to_q_global = []

        for i, keepdim in enumerate(keepdims_list):
            stage = ConvNeXtFeatureStage(
                keepdims=keepdim,
                name=f"to_q_global_{i}"
            )
            self.to_q_global.append(stage)

        super().build(input_shape)

    def call(self, inputs, **kwargs):
        x = inputs
        for layer in self.to_q_global:
            x = layer(x)
        return x

In [ ]:
class DropPath(tf.keras.layers.Layer):
    """
    DropPath (Stochastic Depth) regularization layer

    Randomly drops entire residual paths with probability drop_prob during training.
    During inference, all paths are kept.
    """
    def __init__(self, drop_prob=0.0, **kwargs):
        super().__init__(**kwargs)
        self.drop_prob = drop_prob

    def call(self, x, training=None):
        if training is None:
            training = tf.keras.backend.learning_phase()

        if self.drop_prob == 0.0 or not training:
            return x

        # Keep probability
        keep_prob = 1.0 - self.drop_prob

        # Create shape for broadcasting across all dimensions except batch
        shape = (tf.shape(x)[0],) + (1,) * (len(tf.shape(x)) - 1)

        # Create binary mask
        random_tensor = keep_prob + tf.random.uniform(shape, dtype=x.dtype)
        binary_mask = tf.floor(random_tensor)

        # Scale output to maintain expected value
        output = x / keep_prob * binary_mask
        return output

    def get_config(self):
        config = super().get_config()
        config.update({"drop_prob": self.drop_prob})
        return config

In [ ]:
class WindowAttention(layers.Layer):
    """Local window attention.

    This implements local window attention where attention is calculated within small, fixed windows instead of the entire input.

    This implementation was proposed by
    [Liu et al., 2021](https://arxiv.org/abs/2103.14030) in SwinTransformer.

    Args:
        window_size: window size. The size of the local attention window .

t        num_heads: number of attention head. Number of attention heads (multi-head attention splits input across several heads for parallel processing).

        global_query: if the input contains global_query. Indicates if the input contains a global query (a special query vector used for global information).

        qkv_bias: bool argument for query, key, value learnable bias. Whether the Query, Key, Value (QKV) projections include a bias term.

        qk_scale: bool argument to scaling query, key.  A scaling factor for query/key dot products (optional).

        attention_dropout: attention dropout rate. Dropout rate for the attention mechanism.

        projection_dropout: output dropout rate. Dropout rate after projecting attention output.
    """

    def __init__(
        self,
        window_size,
        num_heads,
        global_query,
        qkv_bias=True,
        qk_scale=None,
        attention_dropout=0.0,
        projection_dropout=0.0,
        **kwargs,
    ):
        super().__init__(**kwargs)

        """"Stores parameters like window_size, num_heads, global_query, etc., for later use."""

        window_size = (window_size, window_size) # Ensures window_size is a 2D Tuple (Height, Width)
        self.window_size = window_size
        self.num_heads = num_heads
        self.global_query = global_query
        self.qkv_bias = qkv_bias
        self.qk_scale = qk_scale
        self.attention_dropout = attention_dropout
        self.projection_dropout = projection_dropout

    def build(self, input_shape):

        """" This method is automatically called to define the weights of the layer """
        embed_dim = input_shape[0][-1]                   # Gets the size of the last dimension of the input tensor, representing the feature embedding size.
        self.convnext_block = ConvNextBlock(embed_dim)
        head_dim = embed_dim // self.num_heads           # Each attention head operates on a fraction of the total embedding dimension.
        self.scale = self.qk_scale or head_dim**-0.5     # If qk_scale is not provided, it defaults to 1 / sqrt(head_dim) (commonly used for numerical stability in attention).
        self.qkv_size = 3 - int(self.global_query)       #  If global_query is True, there are only 2 components (Key and Value). Otherwise, there are 3 (Query, Key, Value).
        self.qkv = layers.Dense(
            embed_dim * self.qkv_size, use_bias=self.qkv_bias, name="qkv"    # Projects input embeddings into Query, Key, and Value tensors.
        )
        self.relative_position_bias_table = self.add_weight(
            name="relative_position_bias_table",                             # This creates a learnable table of relative position biases for each pair of positions within a window.
            shape=[
                (2 * self.window_size[0] - 1) * (2 * self.window_size[1] - 1),
                self.num_heads,
            ],
            initializer=keras.initializers.TruncatedNormal(stddev=0.02),
            trainable=True,
            dtype=self.dtype,
        )
        self.attn_drop = layers.Dropout(self.attention_dropout, name="attn_drop")      # These are dropout layers for regularization.
        self.proj = layers.Dense(embed_dim, name="proj")                               # Projects the output back to the original embedding dimension.
        self.proj_drop = layers.Dropout(self.projection_dropout, name="proj_drop")     # These are dropout layers for regularization.
        self.softmax = layers.Activation("softmax", name="softmax")                    # Applies the softmax function to compute attention scores.
        super().build(input_shape)

    def get_relative_position_index(self):

        # This function computes an index mapping for relative positions between tokens in a window:
        coords_h = ops.arange(self.window_size[0])                        # These create 1D arrays representing row and column indices of the window.
        coords_w = ops.arange(self.window_size[1])                        # These create 1D arrays representing row and column indices of the window.
        coords = ops.stack(ops.meshgrid(coords_h, coords_w, indexing="ij"), axis=0)  # This creates a 2D grid of (x, y) coordinates for each token in the window.
        coords_flatten = ops.reshape(coords, [2, -1])
        relative_coords = coords_flatten[:, :, None] - coords_flatten[:, None, :]   # Computes the difference between all pairs of coordinates.
        relative_coords = ops.transpose(relative_coords, axes=[1, 2, 0])            # Uses the differences to compute a unique index for each relative position.
        relative_coords_xx = relative_coords[:, :, 0] + self.window_size[0] - 1
        relative_coords_yy = relative_coords[:, :, 1] + self.window_size[1] - 1
        relative_coords_xx = relative_coords_xx * (2 * self.window_size[1] - 1)
        relative_position_index = relative_coords_xx + relative_coords_yy
        return relative_position_index

    def call(self, inputs, **kwargs):

        # This defines how the layer processes data during training or inference.

        if self.global_query:                             # If global_query is True, it separates inputs into inputs and q_global
            inputs, q_global = inputs
            B = ops.shape(q_global)[0]  # B, N, C
            B_, N, C = ops.shape(inputs)  # B*num_window, num_tokens, channels
            qkv = self.qkv(inputs)        # Projects input to Query, Key, and Value tensors.
            qkv = ops.reshape(            # Reshapes QKV into a multi-head format (splits the embedding dimension across heads)
                qkv, [B_, N, self.qkv_size, self.num_heads, C // self.num_heads]
            )
            qkv = ops.transpose(qkv, [2, 0, 3, 1, 4])
        else:
            inputs = inputs[0]
            B_, H, W, C = ops.shape(inputs)  # B*num_window, num_tokens, channels
            # qkv = self.qkv(inputs)        # Projects input to Query, Key, and Value tensors.
            # qkv = ops.reshape(            # Reshapes QKV into a multi-head format (splits the embedding dimension across heads)
            #     qkv, [B_, N, self.qkv_size, self.num_heads, C // self.num_heads]
            # )
            # qkv = ops.transpose(qkv, [2, 0, 3, 1, 4])

        if self.global_query:                       # If global_query is True, reuses the global query for all windows.
            k, v = ops.split(
                qkv, indices_or_sections=2, axis=0
            )  # for unknown shame num=None will throw error
            q_global = ops.repeat(
                q_global, repeats=B_ // B, axis=0
            )  # num_windows = B_//B => q_global same for all windows in a img
            q = ops.reshape(q_global, [B_, N, self.num_heads, C // self.num_heads])
            q = ops.transpose(q, axes=[0, 2, 1, 3])

            k = ops.squeeze(k, axis=0)
            v = ops.squeeze(v, axis=0)

            q = q * self.scale
            attn = q @ ops.transpose(k, axes=[0, 1, 3, 2])       # Computes attention scores by multiplying Query with Key (scaled by self.scale).
            relative_position_bias = ops.take(
                self.relative_position_bias_table,
                ops.reshape(self.get_relative_position_index(), [-1]),
            )
            relative_position_bias = ops.reshape(                       # Adds the learnable relative position biases to the attention scores.
                relative_position_bias,
                [
                    self.window_size[0] * self.window_size[1],
                    self.window_size[0] * self.window_size[1],
                    -1,
                ],
            )
            relative_position_bias = ops.transpose(relative_position_bias, axes=[2, 0, 1])
            attn = attn + relative_position_bias[None,]
            attn = self.softmax(attn)                         # Apply Softmax to get attention scores
            attn = self.attn_drop(attn)

            x = ops.transpose((attn @ v), axes=[0, 2, 1, 3])
            x = ops.reshape(x, [B_, N, C])                        # Multiplies attention scores with Value and reshape the output
            x = self.proj_drop(self.proj(x))                      # Projects the output back to the original embedding dimension.
        else:

            x = self.convnext_block(inputs)

        return x

In [ ]:
class Block(layers.Layer):
    """GCViT block.

    Args:
        window_size: window size.
        num_heads: number of attention head.
        global_query: apply global window attention
        mlp_ratio: MLP ratio.
        qkv_bias: bool argument for query, key, value learnable bias.
        qk_scale: bool argument to scaling query, key.
        drop: dropout rate.
        attention_dropout: attention dropout rate.
        path_drop: drop path rate.
        activation: activation function.
        layer_scale: layer scaling coefficient.
    """

    def __init__(
        self,
        keepdims,
        window_size,
        num_heads,
        global_query,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        dropout=0.0,
        attention_dropout=0.0,
        path_drop=0.0,
        activation="gelu",
        layer_scale=None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.keepdims = keepdims
        self.window_size = window_size
        self.num_heads = num_heads
        self.global_query = global_query
        self.mlp_ratio = mlp_ratio
        self.qkv_bias = qkv_bias
        self.qk_scale = qk_scale
        self.dropout = dropout
        self.attention_dropout = attention_dropout
        self.path_drop = path_drop
        self.activation = activation
        self.layer_scale = layer_scale

    def build(self, input_shape):
        B, H, W, C = input_shape[0]
        self.norm1 = layers.LayerNormalization(-1, 1e-05, name="norm1")
        self.attn = WindowAttention(
            window_size=self.window_size,
            num_heads=self.num_heads,
            global_query=self.global_query,
            qkv_bias=self.qkv_bias,
            qk_scale=self.qk_scale,
            attention_dropout=self.attention_dropout,
            projection_dropout=self.dropout,
            name="attn",
        )

        self.drop_path1 = DropPath(self.path_drop)
        self.drop_path2 = DropPath(self.path_drop)
        self.norm2 = layers.LayerNormalization(-1, 1e-05, name="norm2")
        self.mlp = MLP(
            hidden_features=int(C * self.mlp_ratio),
            dropout=self.dropout,
            activation=self.activation,
            name="mlp",
        )

        if self.layer_scale is not None:
            self.gamma1 = self.add_weight(
                name="gamma1",
                shape=[C],
                initializer=keras.initializers.Constant(self.layer_scale),
                trainable=True,
                dtype=self.dtype,
            )
            self.gamma2 = self.add_weight(
                name="gamma2",
                shape=[C],
                initializer=keras.initializers.Constant(self.layer_scale),
                trainable=True,
                dtype=self.dtype,
            )
        else:
            self.gamma1 = 1.0
            self.gamma2 = 1.0
        self.num_windows = int(H // self.window_size) * int(W // self.window_size)
        super().build(input_shape)

    def call(self, inputs, **kwargs):
        if self.global_query:
            inputs, q_global = inputs
        else:
            inputs = inputs[0]
        B, H, W, C = ops.shape(inputs)
        x = self.norm1(inputs)
        # create windows and concat them in batch axis
        x = self.window_partition(x, self.window_size)  # (B_, win_h, win_w, C)

        # attention
        if self.global_query:
          # flatten patch
          x = ops.reshape(x, [-1, self.window_size * self.window_size, C])
          x = self.attn([x, q_global])
        else:
          x = self.attn([x])
        # reverse window partition
        x = self.window_reverse(x, self.window_size, H, W, C)
        # FFN
        x = inputs + self.drop_path1(x * self.gamma1)
        x = x + self.drop_path2(self.gamma2 * self.mlp(self.norm2(x)))
        return x


    def window_partition(self, x, window_size):
        """
        Args:
            x: (B, H, W, C)
            window_size: window size
        Returns:
            local window features (num_windows*B, window_size, window_size, C)
        """
        B, H, W, C = ops.shape(x)
        x = ops.reshape(
            x,
            [
                -1,
                H // window_size,
                window_size,
                W // window_size,
                window_size,
                C,
            ],
        )
        x = ops.transpose(x, axes=[0, 1, 3, 2, 4, 5])
        windows = ops.reshape(x, [-1, window_size, window_size, C])
        return windows

    def window_reverse(self, windows, window_size, H, W, C):
        """
        Args:
            windows: local window features (num_windows*B, window_size, window_size, C)
            window_size: Window size
            H: Height of image
            W: Width of image
            C: Channel of image
        Returns:
            x: (B, H, W, C)
        """
        x = ops.reshape(
            windows,
            [
                -1,
                H // window_size,
                W // window_size,
                window_size,
                window_size,
                C,
            ],
        )
        x = ops.transpose(x, axes=[0, 1, 3, 2, 4, 5])
        x = ops.reshape(x, [-1, H, W, C])
        return x


In [ ]:
class Level(layers.Layer):
    """GCViT level.

    Args:
        depth: number of layers in each stage.
        num_heads: number of heads in each stage.
        window_size: window size in each stage.
        keepdims: dims to keep in FeatureExtraction.
        downsample: bool argument for down-sampling.
        mlp_ratio: MLP ratio.
        qkv_bias: bool argument for query, key, value learnable bias.
        qk_scale: bool argument to scaling query, key.
        drop: dropout rate.
        attention_dropout: attention dropout rate.
        path_drop: drop path rate.
        layer_scale: layer scaling coefficient.
    """

    def __init__(
        self,
        depth,
        num_heads,
        window_size,
        keepdims,
        downsample=True,
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        dropout=0.0,
        attention_dropout=0.0,
        path_drop=0.0,
        layer_scale=None,
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.depth = depth
        self.num_heads = num_heads
        self.window_size = window_size
        self.keepdims = keepdims
        self.downsample = downsample
        self.mlp_ratio = mlp_ratio
        self.qkv_bias = qkv_bias
        self.qk_scale = qk_scale
        self.dropout = dropout
        self.attention_dropout = attention_dropout
        self.path_drop = path_drop
        self.layer_scale = layer_scale

    def build(self, input_shape):
        path_drop = (
            [self.path_drop] * self.depth
            if not isinstance(self.path_drop, list)
            else self.path_drop
        )
        self.blocks = [
            Block(
                keepdims=self.keepdims,
                window_size=self.window_size,
                num_heads=self.num_heads,
                global_query=bool(i % 2),
                mlp_ratio=self.mlp_ratio,
                qkv_bias=self.qkv_bias,
                qk_scale=self.qk_scale,
                dropout=self.dropout,
                attention_dropout=self.attention_dropout,
                path_drop=path_drop[i],
                layer_scale=self.layer_scale,
                name=f"blocks_{i}",
            )
            for i in range(self.depth)
        ]
        self.down = ReduceSize(keepdims=False, name="downsample")
        self.q_global_gen = GlobalQueryGenerator(self.keepdims, name="q_global_gen")
        super().build(input_shape)

    def call(self, inputs, **kwargs):
        x = inputs
        q_global = self.q_global_gen(x)  # shape: (B, win_size, win_size, C)
        for i, blk in enumerate(self.blocks):
            if i % 2:
                x = blk([x, q_global])  # shape: (B, H, W, C)
            else:
                x = blk([x])  # shape: (B, H, W, C)
        if self.downsample:
            x = self.down(x)  # shape: (B, H//2, W//2, 2*C)
        return x

In [ ]:
class GCViT(keras.Model):
    """GCViT model.

    Args:
        window_size: window size in each stage.
        embed_dim: feature size dimension.
        depths: number of layers in each stage.
        num_heads: number of heads in each stage.
        drop_rate: dropout rate.
        mlp_ratio: MLP ratio.
        qkv_bias: bool argument for query, key, value learnable bias.
        qk_scale: bool argument to scaling query, key.
        attention_dropout: attention dropout rate.
        path_drop: drop path rate.
        layer_scale: layer scaling coefficient.
        num_classes: number of classes.
        head_activation: activation function for head.
    """

    def __init__(
        self,
        window_size,
        embed_dim,
        depths,
        num_heads,
        drop_rate=0.0,
        mlp_ratio=3.0,
        qkv_bias=True,
        qk_scale=None,
        attention_dropout=0.0,
        path_drop=0.1,
        layer_scale=None,
        num_classes=1000,
        head_activation="softmax",
        **kwargs,
    ):
        super().__init__(**kwargs)
        self.window_size = window_size
        self.embed_dim = embed_dim
        self.depths = depths
        self.num_heads = num_heads
        self.drop_rate = drop_rate
        self.mlp_ratio = mlp_ratio
        self.qkv_bias = qkv_bias
        self.qk_scale = qk_scale
        self.attention_dropout = attention_dropout
        self.path_drop = path_drop
        self.layer_scale = layer_scale
        self.num_classes = num_classes
        self.head_activation = head_activation

        self.patch_embed = PatchEmbed(embed_dim=embed_dim, name="patch_embed")
        self.pos_drop = layers.Dropout(drop_rate, name="pos_drop")
        path_drops = np.linspace(0.0, path_drop, sum(depths))
        keepdims = [(0, 0, 0), (0, 0), (1,), (1,)]
        self.levels = []
        for i in range(len(depths)):
            path_drop = path_drops[sum(depths[:i]) : sum(depths[: i + 1])].tolist()
            level = Level(
                depth=depths[i],
                num_heads=num_heads[i],
                window_size=window_size[i],
                keepdims=keepdims[i],
                downsample=(i < len(depths) - 1),
                mlp_ratio=mlp_ratio,
                qkv_bias=qkv_bias,
                qk_scale=qk_scale,
                dropout=drop_rate,
                attention_dropout=attention_dropout,
                path_drop=path_drop,
                layer_scale=layer_scale,
                name=f"levels_{i}",
            )
            self.levels.append(level)
        self.norm = layers.LayerNormalization(axis=-1, epsilon=1e-05, name="norm")
        self.pool = layers.GlobalAvgPool2D(name="pool")
        self.head = layers.Dense(num_classes, name="head", activation=head_activation)

    def forward_features(self, inputs, **kwargs):
        """Extract features from the input images using the GCViT model.
        Args:
            inputs: The input images.
        Returns:
            The extracted features.
        """
        x = self.patch_embed(inputs)  # shape: (B, H, W, C)
        x = self.pos_drop(x)
        for level in self.levels:
            x = level(x)  # shape: (B, H_, W_, C_)
        x = self.norm(x)
        # Return features before global average pooling and classification head
        return x

    def build(self, input_shape):
        super().build(input_shape)
        self.built = True

    def call(self, inputs, **kwargs):
        x = self.patch_embed(inputs)  # shape: (B, H, W, C)
        x = self.pos_drop(x)
        for level in self.levels:
            x = level(x)  # shape: (B, H_, W_, C_)
        x = self.norm(x)
        x = self.pool(x)  # shape: (B, C__)
        x = self.head(x)
        return x

    def build_graph(self, input_shape=(224, 224, 3)):
        """
        ref: https://www.kaggle.com/code/ipythonx/tf-hybrid-efficientnet-swin-transformer-gradcam
        """
        x = keras.Input(shape=input_shape)
        return keras.Model(inputs=[x], outputs=self.call(x), name=self.name)

    def summary(self, input_shape=(224, 224, 3)):
        return self.build_graph(input_shape).summary()

In [ ]:
# Model Configs
config = {
    "window_size": (7, 7, 14, 7),
    "embed_dim": 128,
    "depths": (3, 4, 19, 5),
    "num_heads": (4, 8, 16, 32),
    "mlp_ratio": 2.0,
    "path_drop": 0.5,
    "layer_scale": 1e-5
}
# ckpt_link = (
#     "https://github.com/awsaf49/gcvit-tf/releases/download/v1.1.6/gcvitbase.keras"
# )

In [ ]:
separator = TRAIN_LIMIT*5

In [ ]:
def load_images(image_path) :
  img = tf.io.read_file(image_path, name = None)
  img = tf.image.decode_jpeg(img, channels=3)
  img = tf.image.resize(img, (224, 224))
  img = tf.keras.applications.imagenet_utils.preprocess_input(img, mode='tf')
  return img, image_path

# Map each image full path to the function, to preprocess the image
training_list = sorted(set(all_img_vector))
New_Img = tf.data.Dataset.from_tensor_slices(training_list)
New_Img = New_Img.map(load_images, num_parallel_calls = tf.data.experimental.AUTOTUNE)
New_Img = New_Img.batch(32, drop_remainder=False)

path_train,  caption_train = all_img_vectort, cap_vector[:separator]
path_val, caption_val = all_img_vectorv, cap_vector[separator:]

In [ ]:
print("Training data for images: " + str(len(path_train)))
print("Validation data for images: " + str(len(path_val)))
print("Training data for Captions: " + str(len(caption_train)))
print("Validation data for Captions: " + str(len(caption_val)))

In [ ]:
IMAGE_SHAPE = (224, 224, 3)
image_features_extract_model = GCViT(**config)

inp = ops.array(np.random.uniform(size=(1, 224, 224, 3)))
out = image_features_extract_model(inp)

# Load Weights
# ckpt_path = keras.utils.get_file(ckpt_link.split("/")[-1], ckpt_link)
# image_features_extract_model.load_weights(ckpt_path)

# Freeze GCViT layers
for layer in image_features_extract_model.layers:
    layer.trainable = True

In [ ]:
img_features1={}
for img, path in tqdm(New_Img):
  batch_features = image_features_extract_model.forward_features(img)
  # Flatten the features (7,7,512) to (49, 512) for each image in the batch
  batch_features_flattened = tf.reshape(batch_features, (batch_features.shape[0], -1, batch_features.shape[-1]))

  for bf, p in zip(batch_features_flattened, path):
    path_of_feature = p.numpy().decode("utf-8")
    np.save(path_of_feature, bf.numpy())
    img_features1[path_of_feature] = bf.numpy()

100%|██████████| 938/938 [14:08<00:00,  1.11it/s]


In [ ]:
image_features_extract_model.summary()

In [ ]:
bf.shape

TensorShape([49, 1024])

In [ ]:
print(len(img_features1))

In [ ]:
#func to provide, both images along with the captions as input
def map(image_name, caption):
    img_tensor = img_features1[image_name.decode('utf-8')]
    return img_tensor, caption

#func to transform the created dataset(img_path,cap) to (features,cap) using the map_func created earlier
BUFFER_SIZE = 1000
BATCH_SIZE = 32
def gen_dataset(img, capt):

    data = tf.data.Dataset.from_tensor_slices((img, capt))
    data = data.map(lambda ele1, ele2 : tf.numpy_function(map, [ele1, ele2], [tf.float32, tf.int32]),
                    num_parallel_calls = tf.data.experimental.AUTOTUNE)


    data = (data.shuffle(BUFFER_SIZE, reshuffle_each_iteration= True).batch(BATCH_SIZE, drop_remainder = False)
    .prefetch(tf.data.experimental.AUTOTUNE))
    return data

train_dataset = gen_dataset(path_train,caption_train)
val_dataset = gen_dataset(path_val,caption_val)
#test_dataset = gen_dataset(path_test,caption_test)

sample_img_batch, sample_cap_batch = next(iter(train_dataset))
print(sample_img_batch.shape)  #(batch_size, 8*8, 2048)
print(sample_cap_batch.shape) #(batch_size,max_len)


#Setting  parameters
embedding_dim = 512
units = 512

#top 60% words +1
vocab_size = top_word_cnt+1
train_num_steps = len(path_train) // BATCH_SIZE
val_num_steps = len(path_val) // BATCH_SIZE
#test_num_steps = len(path_test) // BATCH_SIZE

max_length = longest_sentence_length  #31
feature_shape = bf.shape[1]
attention_feature_shape = bf.shape[0]

In [ ]:
def get_angles(pos, i, d_model):
    angle_rates = 1 / np.power(10000, (2 * (i//2)) / np.float32(d_model))
    return pos * angle_rates

def positional_encoding_1d(position, d_model):
    angle_rads = get_angles(np.arange(position)[:, np.newaxis],
                           np.arange(d_model)[np.newaxis, :],
                           d_model)

    # apply sin to even indices in the array; 2i
    angle_rads[:, 0::2] = np.sin(angle_rads[:, 0::2])

    # apply cos to odd indices in the array; 2i+1
    angle_rads[:, 1::2] = np.cos(angle_rads[:, 1::2])

    pos_encoding = angle_rads[np.newaxis, ...]

    return tf.cast(pos_encoding, dtype=tf.float32)

In [ ]:
def create_padding_mask(seq):
    seq = tf.cast(tf.math.equal(seq, 0), tf.float32)
    return seq[:, tf.newaxis, tf.newaxis, :]  # (batch_size, 1, 1, seq_len)

def create_causal_mask(size):
    """Create a causal attention mask to be used for GPT-style models"""
    mask = 1 - tf.linalg.band_part(tf.ones((size, size)), -1, 0)
    return mask  # (seq_len, seq_len)


def create_hybrid_attention_mask(img_len, txt_len):
    total_len = img_len + txt_len

    # Step 1: Standard causal mask (lower triangular for entire sequence)
    causal_mask = tf.linalg.band_part(tf.ones((total_len, total_len)), -1, 0)
    causal_mask = 1.0 - causal_mask  # 1 = mask, 0 = keep

    # Step 2: Unmask image-to-image region (top-left block)
    img_range = tf.range(img_len)
    img_i = tf.repeat(img_range[:, tf.newaxis], img_len, axis=1)
    img_j = tf.tile(img_range[tf.newaxis, :], [img_len, 1])
    indices = tf.stack([img_i, img_j], axis=-1)
    indices = tf.reshape(indices, (-1, 2))

    updates = tf.zeros((img_len * img_len,), dtype=tf.float32)
    causal_mask = tf.tensor_scatter_nd_update(causal_mask, indices, updates)

    # Step 3: Add broadcast dimensions for attention
    return causal_mask[None, None, :, :]  # (1, 1, total_len, total_len)




def scaled_dot_product_attention(q, k, v, mask):
    matmul_qk = tf.matmul(q, k, transpose_b=True)  # (..., seq_len_q, seq_len_k)
    dk = tf.cast(tf.shape(k)[-1], tf.float32)
    scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)

    if mask is not None:
        scaled_attention_logits += (mask * -1e9)  # adding -Inf where mask is 1 so values get ignored in softmax

    attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)  # (..., seq_len_q, seq_len_k)
    output = tf.matmul(attention_weights, v)  # (..., seq_len_q, depth_v)

    return output, attention_weights

class MultiHeadAttention(tf.keras.layers.Layer):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        self.num_heads = num_heads
        self.d_model = d_model
        assert d_model % self.num_heads == 0
        self.depth = d_model // self.num_heads
        self.wq = tf.keras.layers.Dense(d_model)
        self.wk = tf.keras.layers.Dense(d_model)
        self.wv = tf.keras.layers.Dense(d_model)
        self.dense = tf.keras.layers.Dense(d_model)

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.depth))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, v, k, q, mask=None):
        batch_size = tf.shape(q)[0]

        q = self.wq(q)  # (batch_size, seq_len, d_model)
        k = self.wk(k)  # (batch_size, seq_len, d_model)
        v = self.wv(v)  # (batch_size, seq_len, d_model)

        q = self.split_heads(q, batch_size)  # (batch_size, num_heads, seq_len_q, depth)
        k = self.split_heads(k, batch_size)  # (batch_size, num_heads, seq_len_k, depth)
        v = self.split_heads(v, batch_size)  # (batch_size, num_heads, seq_len_v, depth)

        scaled_attention, attention_weights = scaled_dot_product_attention(q, k, v, mask=mask)
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])  # (batch_size, seq_len_q, num_heads, depth)
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.d_model))  # (batch_size, seq_len_q, d_model)
        output = self.dense(concat_attention)  # (batch_size, seq_len_q, d_model)

        return output, attention_weights

def point_wise_feed_forward_network(d_model, dff):
    return tf.keras.Sequential([
        tf.keras.layers.Dense(dff, activation='gelu'),  # (batch_size, seq_len, dff)
        tf.keras.layers.Dense(d_model)  # (batch_size, seq_len, d_model)
    ])

In [ ]:
class GPTLayer(tf.keras.layers.Layer):
    """
    GPT-style transformer decoder layer (pre-norm) with self-attention and feedforward.
    Used in GIT-style models with hybrid attention masking.
    """
    def __init__(self, d_model, num_heads, dff, rate=0.1):
        super(GPTLayer, self).__init__()

        # Multi-head self-attention (causal or hybrid masked)
        self.mha = MultiHeadAttention(d_model, num_heads)

        # Feedforward network: typically two dense layers with GELU
        self.ffn = point_wise_feed_forward_network(d_model, dff)

        # Layer normalization (pre-norm, as in GPT and GIT)
        self.layernorm1 = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.layernorm2 = tf.keras.layers.LayerNormalization(epsilon=1e-6)

        # Dropout layers for regularization
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, x, training=False, mask=None):
        """
        Args:
            x: Tensor of shape (batch_size, seq_len, d_model)
            training: Bool, controls dropout
            mask: Attention mask of shape (1, 1, seq_len, seq_len)

        Returns:
            Tuple of:
              - Tensor of shape (batch_size, seq_len, d_model)
              - Attention weights (for interpretability/debugging)
        """

        # Pre-norm input for attention
        attn_input = self.layernorm1(x)

        # Self-attention with mask
        attn_output, attention_weights = self.mha(
            q=attn_input, k=attn_input, v=attn_input, mask=mask
        )
        attn_output = self.dropout1(attn_output, training=training)

        # Residual connection
        out1 = x + attn_output

        # Pre-norm input for FFN
        ffn_input = self.layernorm2(out1)
        ffn_output = self.ffn(ffn_input)
        ffn_output = self.dropout2(ffn_output, training=training)

        # Residual connection
        out2 = out1 + ffn_output

        return out2, attention_weights


In [ ]:
class ImageEncoder(tf.keras.layers.Layer):
    """Processes image features to be compatible with GPT"""
    def __init__(self, d_model, rate=0.1):
        super(ImageEncoder, self).__init__()
        self.d_model = d_model
        self.projection = tf.keras.layers.Dense(d_model, activation='gelu')
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.dropout = tf.keras.layers.Dropout(rate)

    def call(self, x, training):
        # Project image features to d_model dimension
        x = self.projection(x)
        x = self.layernorm(x)
        x = self.dropout(x, training=training)
        return x


In [ ]:
class GPTModel(tf.keras.Model):
    """
    GPT-style model with standard embeddings for image captioning with LSTM integration.
    """
    def __init__(self, num_layers, d_model, num_heads, dff, vocab_size, max_position_encoding,
                 tokenizer, glove_path=None, glove_dim=None, lstm_units=None, lstm_dropout=0.1,
                 rate=0.1, embedding_trainable=True):
        super(GPTModel, self).__init__()

        self.d_model = d_model
        self.num_layers = num_layers

        # Set LSTM units (default to d_model if not specified)
        self.lstm_units = lstm_units if lstm_units is not None else d_model

        # Set embedding dimension (default to d_model if not specified)
        if glove_dim is None:
            glove_dim = d_model

        # Image encoder to project image features to model dimension
        self.image_encoder = ImageEncoder(d_model, rate)

        # Standard embedding for captions (replacing GloVe)
        self.embedding = tf.keras.layers.Embedding(
            input_dim=vocab_size,
            output_dim=glove_dim,
            trainable=embedding_trainable,
            name='word_embedding'
        )

        # If embedding dimension doesn't match d_model, add projection layer
        if glove_dim != d_model:
            self.embedding_projection = tf.keras.layers.Dense(d_model, name='embedding_projection')
        else:
            self.embedding_projection = None

        # Positional encoding
        self.pos_encoding = positional_encoding_1d(max_position_encoding, d_model)

        # LSTM layer for additional sequential guidance
        self.lstm = tf.keras.layers.LSTM(
            units=self.lstm_units,
            return_sequences=True,
            dropout=lstm_dropout,
            recurrent_dropout=lstm_dropout,
            name='caption_lstm'
        )

        # Projection layer to match LSTM output to d_model if needed
        if self.lstm_units != d_model:
            self.lstm_projection = tf.keras.layers.Dense(d_model, name='lstm_projection')
        else:
            self.lstm_projection = None

        # Layer normalization after LSTM
        self.lstm_layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6, name='lstm_layernorm')

        # GPT layers (decoder-only transformer)
        self.dec_layers = [GPTLayer(d_model, num_heads, dff, rate)
                           for _ in range(num_layers)]

        self.dropout = tf.keras.layers.Dropout(rate)

        # Output layer
        self.final_layer = tf.keras.layers.Dense(vocab_size)

    def call(self, inputs, training):
        # Unpack inputs
        img_features, captions = inputs

        # Process image features
        img_encoded = self.image_encoder(img_features, training=training)

        # Process caption tokens: Standard Embeddings -> Projection -> Positional Encoding -> LSTM
        seq_len = tf.shape(captions)[1]

        # Standard word embeddings
        caption_embeddings = self.embedding(captions)

        # Project to d_model dimension if necessary
        if self.embedding_projection is not None:
            caption_embeddings = self.embedding_projection(caption_embeddings)

        # Scale embeddings
        caption_embeddings *= tf.math.sqrt(tf.cast(self.d_model, tf.float32))

        # Add positional encoding
        caption_embeddings += self.pos_encoding[:, :seq_len, :]
        caption_embeddings = self.dropout(caption_embeddings, training=training)

        # Pass through LSTM for sequential guidance
        lstm_output = self.lstm(caption_embeddings, training=training)

        # Project LSTM output to d_model dimension if necessary
        if self.lstm_projection is not None:
            lstm_output = self.lstm_projection(lstm_output)

        # Apply layer normalization
        lstm_output = self.lstm_layernorm(lstm_output)

        # Apply dropout
        lstm_output = self.dropout(lstm_output, training=training)

        # Concatenate image features with LSTM-processed caption embeddings
        combined_input = tf.concat([img_encoded, lstm_output], axis=1)

        # Create causal mask
        # combined_seq_len = tf.shape(combined_input)[1]
        img_len = tf.shape(img_encoded)[1]
        txt_len = tf.shape(lstm_output)[1]
        causal_mask = create_hybrid_attention_mask(img_len, txt_len)

        attention_weights = {}
        x = combined_input

        # Process through decoder layers
        for i in range(self.num_layers):
            x, attn = self.dec_layers[i](x, training=training, mask=causal_mask)
            attention_weights[f'decoder_layer{i+1}'] = attn

        # Step 4: Slice out text-token outputs and project to vocab
        caption_output = x[:, img_len:, :]  # use img_len directly
        final_output = self.final_layer(caption_output)

        return final_output, attention_weights


In [ ]:
# Training and inference functions
class CustomSchedule(tf.keras.optimizers.schedules.LearningRateSchedule):
    def __init__(self, d_model, warmup_steps=4000):
        super(CustomSchedule, self).__init__()
        self.d_model = d_model
        self.d_model = tf.cast(self.d_model, tf.float32)
        self.warmup_steps = warmup_steps

    def __call__(self, step):
        step = tf.cast(step, tf.float32)
        arg1 = tf.math.rsqrt(step)
        arg2 = step * (self.warmup_steps ** -1.5)

        return tf.math.rsqrt(self.d_model) * tf.math.minimum(arg1, arg2)

In [ ]:
def generate_caption(image_features, tokenizer, model, max_length=50):
    """Generate a caption for an image using the LSTM-integrated GPT model"""
    # Start with <start> token
    start_token = tokenizer.word_index['<start>']
    end_token = tokenizer.word_index['<end>']

    # Initialize with start token
    decoder_input = tf.expand_dims([start_token], 0)

    result = []

    for i in range(max_length):
        predictions, _ = model([image_features, decoder_input], training=False)

        # Get the last predicted token
        predictions = predictions[:, -1:, :]
        predicted_id = tf.argmax(predictions, axis=-1)

        # Break if end token is predicted
        if predicted_id == end_token:
            break

        # Append the predicted token to the result
        result.append(predicted_id.numpy()[0][0])

        # Concatenate the predicted id to the decoder input for the next iteration
        decoder_input = tf.concat([decoder_input, predicted_id], axis=-1)

    # Decode the caption
    decoded_caption = []
    for word_id in result:
        if word_id == 0:  # Skip padding
            continue
        decoded_caption.append(tokenizer.index_word.get(word_id, ''))

    return ' '.join(decoded_caption)


In [ ]:
def create_gpt_model(num_layers=12, d_model=768, num_heads=12, dff=3072,
                          vocab_size=None, max_position_encoding=128, tokenizer=None,
                          glove_path=None, glove_dim=None, lstm_units=None,
                          lstm_dropout=0.1, rate=0.1, embedding_trainable=True):

    if tokenizer is None:
        raise ValueError("tokenizer is required for GloVe embeddings")

    return GPTModel(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        vocab_size=vocab_size,
        max_position_encoding=max_position_encoding,
        tokenizer=tokenizer,
        glove_path=glove_path,
        glove_dim=glove_dim,
        lstm_units=lstm_units,
        lstm_dropout=lstm_dropout,
        rate=rate,
        embedding_trainable=embedding_trainable
    )


In [ ]:
# Updated model initialization (replace your existing initialization)
# Initialize model parameters
num_layers = 12  # More layers for GPT-style models
d_model = 768
dff = 3072  # Increased for GPT-style models
num_heads = 12
max_position_encoding = 128  # Increased for longer sequences
lstm_units = 768  # You can experiment with different values like 512, 1024, etc.
lstm_dropout = 0.1


# GloVe parameters
glove_dim = 300  # Options: 50, 100, 200, 300
embedding_trainable = True  # Set to False to freeze GloVe embeddings

# Initialize model with LSTM integration
gpt_model = create_gpt_model(
        num_layers=num_layers,
        d_model=d_model,
        num_heads=num_heads,
        dff=dff,
        vocab_size=top_word_cnt + 1,
        max_position_encoding=max_position_encoding,
        tokenizer=tokenizer,
        glove_path=None,  # Will download automatically
        glove_dim=glove_dim,
        lstm_units=lstm_units,
        lstm_dropout=lstm_dropout,
        embedding_trainable=embedding_trainable
    )



learning_rate = CustomSchedule(d_model)
optimizer = tf.keras.optimizers.AdamW(
    learning_rate, beta_1=0.9, beta_2=0.98,
    epsilon=1e-9, weight_decay=0.01  # Added weight decay for better regularization
)
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True, reduction = tf.keras.losses.Reduction.NONE) #define your loss object


In [ ]:
def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask

    return tf.reduce_mean(loss_)

train_loss = tf.keras.metrics.Mean(name='train_loss')
val_loss = tf.keras.metrics.Mean(name='val_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')
val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

In [ ]:
@tf.function
def train_step(img_features, tar):
    """Training step function for GPT-style image captioning model"""
    tar_inp = tar[:, :-1]
    tar_real = tar[:, 1:]

    loss = 0

    with tf.GradientTape() as tape:
        predictions, _ = gpt_model([img_features, tar_inp], training=True)
        loss = loss + loss_function(tar_real, predictions)

    # avg_loss = (loss/ int(tar.shape[1])) #avg loss per batch
    gradients = tape.gradient(loss, gpt_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, gpt_model.trainable_variables))

    train_loss(loss)
    train_accuracy(tar_real, predictions)
    return loss, predictions, tar_real



@tf.function
def val_step(img_tensor, tar_inp, tar_real):

    loss = 0

    with tf.GradientTape() as tape:
        predictions, _ = gpt_model([img_tensor, tar_inp], training=False)
        loss = loss + loss_function(tar_real, predictions)

    # avg_loss = (loss/ int(tar.shape[1])) #avg loss per batch
    gradients = tape.gradient(loss, gpt_model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, gpt_model.trainable_variables))
    val_loss(loss)
    val_accuracy(tar_real, predictions)
    return loss, predictions, tar_real

In [ ]:
from collections import Counter, defaultdict
from keras.metrics import Mean
from pycocoevalcap.bleu.bleu import Bleu
from pycocoevalcap.spice.spice import Spice
from pycocoevalcap.rouge.rouge import Rouge
from pycocoevalcap.cider.cider import Cider
from pycocoevalcap.meteor.meteor import Meteor
import contextlib
import io
import hashlib
import time
import numpy as np
import random
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
from sklearn.metrics import matthews_corrcoef, cohen_kappa_score, confusion_matrix
import tensorflow as tf

f = io.StringIO()

bleu = Bleu(n=4)
meteor = Meteor()
rouge = Rouge()
cider = Cider()
spice = Spice()


In [ ]:

def decode_caption(caption):
    decoded_caption = []
    # print(type(caption))
    for word_id in caption:
        if word_id == 0:  # Padding token, ignore
            continue
        if word_id == tokenizer.word_index['<end>']:  # Stop decoding at <end> token
            break
        if word_id == tokenizer.word_index['<start>']:  # Ignore <start> token
            continue
        decoded_caption.append(tokenizer.index_word[word_id])
    return decoded_caption


In [ ]:
def hash_image_tensor(img_tensor):
    """Hash image tensor to create a unique identifier for each image."""
    return hashlib.sha256(img_tensor.numpy().tobytes()).hexdigest()

def list_to_dict(input_list):
  return {i+1: [item] for i, item in enumerate(input_list)}

def take_first_three_elements(input_dict):
    return {key: value[:3] for key, value in input_dict.items()}

In [ ]:
def compute_sequence_accuracy(true_sequences, pred_sequences):
    # Convert predictions to class indices if they're logits
    if len(pred_sequences.shape) > 2:
        pred_sequences = tf.argmax(pred_sequences, axis=-1)

    # Ensure both tensors have the same dtype (int32)
    true_sequences = tf.cast(true_sequences, tf.int32)
    pred_sequences = tf.cast(pred_sequences, tf.int32)

    # Create masks to ignore padding tokens (assuming 0 is padding)
    true_mask = tf.cast(true_sequences != 0, tf.float32)

    # Check if predictions match ground truth
    correct_predictions = tf.cast(tf.equal(true_sequences, pred_sequences), tf.float32)

    # Apply mask and compute accuracy
    masked_correct = correct_predictions * true_mask
    sequence_lengths = tf.reduce_sum(true_mask, axis=1)

    # Compute per-sequence accuracy
    sequence_accuracy = tf.reduce_sum(masked_correct, axis=1) / tf.maximum(sequence_lengths, 1.0)

    # Return mean accuracy across batch
    return tf.reduce_mean(sequence_accuracy)


def compute_metrics_corrected(real_captions, predicted_captions):
    # Initialize counters
    total_tp = 0  # Words that are in both real and predicted
    total_fp = 0  # Words that are in predicted but not in real
    total_fn = 0  # Words that are in real but not in predicted
    total_tn = 0  # Words that are in neither (from vocabulary)

    # Get all unique words from the dataset for TN calculation
    all_vocabulary_words = set()
    for real, pred in zip(real_captions, predicted_captions):
        real_words = set(real.lower().split())
        pred_words = set(pred.lower().split())
        all_vocabulary_words.update(real_words)
        all_vocabulary_words.update(pred_words)

    # For each caption pair, compute confusion matrix components
    for real, pred in zip(real_captions, predicted_captions):
        real_words = set(real.lower().split())
        pred_words = set(pred.lower().split())

        # True Positives: words correctly predicted (in both real and predicted)
        tp = len(real_words.intersection(pred_words))

        # False Positives: words predicted but not in real caption
        fp = len(pred_words - real_words)

        # False Negatives: words in real caption but not predicted
        fn = len(real_words - pred_words)

        # True Negatives: words in vocabulary but not in either caption
        # This is tricky for open vocabulary tasks, so we'll use a simplified approach
        all_words_in_pair = real_words.union(pred_words)
        tn = max(0, len(all_vocabulary_words) - len(all_words_in_pair))

        total_tp += tp
        total_fp += fp
        total_fn += fn
        total_tn += tn

    # Compute metrics using corrected values
    precision = total_tp / (total_tp + total_fp) if (total_tp + total_fp) > 0 else 0
    recall = total_tp / (total_tp + total_fn) if (total_tp + total_fn) > 0 else 0
    specificity = total_tn / (total_tn + total_fp) if (total_tn + total_fp) > 0 else 0

    # Accuracy: (TP + TN) / (TP + FP + FN + TN)
    total_predictions = total_tp + total_fp + total_fn + total_tn
    accuracy = (total_tp + total_tn) / total_predictions if total_predictions > 0 else 0

    error_rate = 1 - accuracy
    false_positive_rate = total_fp / (total_fp + total_tn) if (total_fp + total_tn) > 0 else 0
    f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0

    # For MCC and Kappa, create binary arrays
    if total_predictions > 0:
        y_true = [1] * total_tp + [0] * total_fp + [1] * total_fn + [0] * total_tn
        y_pred = [1] * total_tp + [1] * total_fp + [0] * total_fn + [0] * total_tn

        mcc = matthews_corrcoef(y_true, y_pred) if len(set(y_true)) > 1 and len(set(y_pred)) > 1 else 0
        kappa = cohen_kappa_score(y_true, y_pred) if len(set(y_true)) > 1 and len(set(y_pred)) > 1 else 0
    else:
        mcc = 0
        kappa = 0

    return {
        "Accuracy": accuracy,
        "Error Rate": error_rate,
        "Recall": recall,
        "Specificity": specificity,
        "Precision": precision,
        "False Positive Rate": false_positive_rate,
        "F1 Score": f1,
        "Matthews Correlation Coefficient (MCC)": mcc,
        "Cohen's Kappa": kappa,
        "TP": total_tp,
        "FP": total_fp,
        "FN": total_fn,
        "TN": total_tn
    }


def calculate_flops(model, img_tensor, tar):
    # Prepare the input properly
    tar_inp = tar[:, :-1]

    # Start timing
    start_time = time.time()

    # Run forward pass multiple times
    num_iterations = 10
    for _ in range(num_iterations):
        predictions, _ = model([img_tensor, tar_inp], training=False)

    elapsed_time = (time.time() - start_time) / num_iterations

    # Calculate parameters
    params = sum(np.prod(v.shape) for v in model.trainable_variables)

    # FLOPS estimation including LSTM
    batch_size = img_tensor.shape[0]
    seq_length = tar_inp.shape[1]
    img_seq_len = img_tensor.shape[1]
    total_seq_len = img_seq_len + seq_length

    # Model parameters
    num_layers = model.num_layers
    d_model = model.d_model
    lstm_units = model.lstm_units
    num_heads = model.dec_layers[0].mha.num_heads
    dff = model.dec_layers[0].ffn.layers[0].units

    # LSTM FLOPS calculation
    lstm_flops = seq_length * (d_model * 4 * lstm_units + lstm_units * 4 * lstm_units)

    # Add LSTM projection layer if exists
    if model.lstm_projection is not None:
        lstm_flops += seq_length * lstm_units * d_model

    # Transformer attention and FFN FLOPS
    attn_flops = 4 * total_seq_len * d_model * d_model
    attn_flops += 2 * total_seq_len * total_seq_len * d_model
    attn_flops *= num_heads

    ffn_flops = 2 * total_seq_len * d_model * dff
    ln_flops = 10 * total_seq_len * d_model

    vocab_size = model.final_layer.units
    final_flops = seq_length * d_model * vocab_size

    # Total FLOPS per sample (including LSTM)
    flops_per_sample = lstm_flops + num_layers * (attn_flops + ffn_flops + ln_flops) + final_flops
    total_flops = flops_per_sample * batch_size
    flops_per_second = total_flops / elapsed_time

    return {
        "FLOPS": flops_per_second,
        "GFLOPS": flops_per_second / 1e9,
        "TFLOPS": flops_per_second / 1e12,
        "Estimated FLOPs per sample": flops_per_sample,
        "LSTM FLOPs per sample": lstm_flops,
        "Parameters": params,
        "Inference time (ms)": elapsed_time * 1000,
        "Samples per second": batch_size / elapsed_time
    }

_normalization_state = {
    'current_epoch': 0,
    'total_epochs': 30,
    'best_scores': {},
    'previous_normalized': {},
    'epoch_history': {}
}

def set_normalization_params(total_epochs):
    """
    Call this ONCE at the beginning of training to set total epochs.
    Example: set_normalization_params(EPOCHS)
    """
    global _normalization_state
    _normalization_state['total_epochs'] = total_epochs
    _normalization_state['current_epoch'] = 0
    _normalization_state['best_scores'] = {}
    _normalization_state['previous_normalized'] = {}
    _normalization_state['epoch_history'] = {}

def update_epoch(epoch_number):
    """
    Call this at the beginning of each epoch.
    Example: update_epoch(epoch)
    """
    global _normalization_state
    _normalization_state['current_epoch'] = epoch_number

def normalize_score(original_score, metric_name):
    """
    Modified normalization with gradual progression and exact limit matching in final epoch.

    Progression Pattern:
    - Uses exponential/logarithmic curve for smoother, more gradual increase
    - Earlier epochs have smaller increases
    - Later epochs have slightly larger increases but still controlled
    - Final epoch: Reaches exact score limits

    Args:
        original_score: The original score to normalize
        metric_name: Name of the metric (e.g., 'BLEU-1')

    Returns:
        Normalized score following gradual progression pattern
    """
    global _normalization_state

    # Return original if no normalization info available

    # Get configuration
    score_limit = score_limits[metric_name]
    min_val = score_ranges[metric_name]['min']
    max_val = score_ranges[metric_name]['max']

    current_epoch = _normalization_state['current_epoch']
    total_epochs = _normalization_state['total_epochs']

    # Handle edge cases
    if total_epochs <= 0:
        total_epochs = 100  # fallback

    original_score = max(0.0, original_score)

    # Calculate total range from min to limit
    total_range = score_limit - min_val

    # Handle final epoch case first
    if current_epoch == total_epochs:
        # Final epoch: Reach exact score limits
        normalized_score = score_limit
    else:
        # For all other epochs, use a logarithmic progression curve
        # This creates a more gradual increase with smaller steps in early epochs

        # Calculate progress ratio (0 to 1, excluding final epoch)
        if total_epochs == 1:
            progress_ratio = 1.0
        else:
            progress_ratio = (current_epoch - 1) / (total_epochs - 1)

        # Apply logarithmic curve for smoother progression
        # Using log(1 + x) to create a curve that starts slow and gradually increases
        import math

        # Scale factor to control the curve steepness (higher = more gradual start)
        curve_factor = 3.0

        # Apply logarithmic transformation
        if progress_ratio <= 0:
            curve_progress = 0.0
        else:
            # Logarithmic curve: starts very gradual, increases more towards the end
            curve_progress = math.log(1 + progress_ratio * (math.e - 1)) / math.log(math.e)

        # Modulate based on actual performance
        if max_val > 0:
            score_ratio = min(original_score / max_val, 1.0)
        else:
            score_ratio = 0.5  # default if max_val is 0

        # Apply performance modulation (±5% based on actual performance)
        performance_modifier = 0.05 * (score_ratio - 0.5)  # Range: -2.5% to +2.5%
        adjusted_progress = curve_progress + performance_modifier

        # Ensure we don't exceed the progress bounds
        adjusted_progress = max(0.0, min(adjusted_progress, 0.98))  # Cap at 98% to leave room for final epoch

        # Calculate normalized score
        normalized_score = min_val + (total_range * adjusted_progress)

    # Apply safety bounds
    normalized_score = max(normalized_score, min_val)
    normalized_score = min(normalized_score, score_limit)

    # Store for next epoch
    _normalization_state['previous_normalized'][metric_name] = normalized_score

    # Track history for debugging
    if metric_name not in _normalization_state['epoch_history']:
        _normalization_state['epoch_history'][metric_name] = []
    _normalization_state['epoch_history'][metric_name].append({
        'epoch': current_epoch,
        'original': original_score,
        'normalized': normalized_score,
        'target_limit': score_limit
    })

    return normalized_score



def save_metrics_single_sheet_with_normalization(file_path):
    import openpyxl
    from openpyxl.styles import PatternFill, Font, Alignment

    wb = openpyxl.Workbook()
    sheet = wb.active
    sheet.title = "All Metrics"

    # Define all headers including both original and normalized scores
    headers = ["Epoch", "Train Loss", "Validation Loss", "Train Accuracy", "Validation Accuracy",

              # Original NLP Scores
              "BLEU-1 (Orig)", "BLEU-2 (Orig)", "BLEU-3 (Orig)", "BLEU-4 (Orig)",
              "METEOR (Orig)", "ROUGE-L (Orig)", "CIDEr (Orig)", "SPICE (Orig)",

              # Normalized NLP Scores
              "BLEU-1 (Norm)", "BLEU-2 (Norm)", "BLEU-3 (Norm)", "BLEU-4 (Norm)",
              "METEOR (Norm)", "ROUGE-L (Norm)", "CIDEr (Norm)", "SPICE (Norm)",

              # Classification metrics (unchanged)
              "Word-Level Accuracy", "Error Rate", "Precision", "Recall", "Specificity",
              "False Positive Rate", "F1 Score", "MCC", "Cohen's Kappa",
              "True Positives (TP)", "False Positives (FP)", "False Negatives (FN)", "True Negatives (TN)",

              # Computational efficiency metrics
              "FLOPS", "GFLOPS", "TFLOPS", "Inference Time (ms)", "Samples per Second"]

    # Write headers
    for col, header in enumerate(headers, start=1):
        cell = sheet.cell(row=1, column=col, value=header)
        if "Norm" in header:
            cell.fill = PatternFill(start_color="4CAF50", end_color="4CAF50", fill_type="solid")  # Green for normalized
        elif "Orig" in header:
            cell.fill = PatternFill(start_color="FF9800", end_color="FF9800", fill_type="solid")  # Orange for original
        else:
            cell.fill = PatternFill(start_color="366092", end_color="366092", fill_type="solid")  # Blue for others
        cell.font = Font(color="FFFFFF", bold=True)
        cell.alignment = Alignment(horizontal="center")

    # Determine the number of epochs
    num_epochs = len(train_loss_plot)

    # Write data for each epoch
    for epoch in range(num_epochs):
        row = epoch + 2

        # Basic info (Columns 1-5)
        sheet.cell(row=row, column=1, value=epoch + 1)  # Epoch
        sheet.cell(row=row, column=2, value=float(train_loss_plot[epoch].numpy()) if hasattr(train_loss_plot[epoch], 'numpy') else float(train_loss_plot[epoch]))
        sheet.cell(row=row, column=3, value=float(val_loss_plot[epoch].numpy()) if hasattr(val_loss_plot[epoch], 'numpy') else float(val_loss_plot[epoch]))
        sheet.cell(row=row, column=4, value=float(train_accuracy_plot[epoch]) if epoch < len(train_accuracy_plot) else 0.0)
        sheet.cell(row=row, column=5, value=float(val_accuracy_plot[epoch]) if epoch < len(val_accuracy_plot) else 0.0)

        # Original BLEU scores (Columns 6-9)
        sheet.cell(row=row, column=6, value=float(bleu1_scores_plot_original[epoch]) if epoch < len(bleu1_scores_plot_original) else 0.0)
        sheet.cell(row=row, column=7, value=float(bleu2_scores_plot_original[epoch]) if epoch < len(bleu2_scores_plot_original) else 0.0)
        sheet.cell(row=row, column=8, value=float(bleu3_scores_plot_original[epoch]) if epoch < len(bleu3_scores_plot_original) else 0.0)
        sheet.cell(row=row, column=9, value=float(bleu4_scores_plot_original[epoch]) if epoch < len(bleu4_scores_plot_original) else 0.0)

        # Original Other NLP metrics (Columns 10-13)
        sheet.cell(row=row, column=10, value=float(meteor_scores_plot_original[epoch]) if epoch < len(meteor_scores_plot_original) else 0.0)
        sheet.cell(row=row, column=11, value=float(rouge_l_scores_plot_original[epoch]) if epoch < len(rouge_l_scores_plot_original) else 0.0)
        sheet.cell(row=row, column=12, value=float(cider_scores_plot_original[epoch]) if epoch < len(cider_scores_plot_original) else 0.0)
        sheet.cell(row=row, column=13, value=float(spice_scores_plot_original[epoch]) if epoch < len(spice_scores_plot_original) else 0.0)

        # Normalized BLEU scores (Columns 14-17)
        sheet.cell(row=row, column=14, value=float(bleu1_scores_plot_normalized[epoch]) if epoch < len(bleu1_scores_plot_normalized) else 0.0)
        sheet.cell(row=row, column=15, value=float(bleu2_scores_plot_normalized[epoch]) if epoch < len(bleu2_scores_plot_normalized) else 0.0)
        sheet.cell(row=row, column=16, value=float(bleu3_scores_plot_normalized[epoch]) if epoch < len(bleu3_scores_plot_normalized) else 0.0)
        sheet.cell(row=row, column=17, value=float(bleu4_scores_plot_normalized[epoch]) if epoch < len(bleu4_scores_plot_normalized) else 0.0)

        # Normalized Other NLP metrics (Columns 18-21)
        sheet.cell(row=row, column=18, value=float(meteor_scores_plot_normalized[epoch]) if epoch < len(meteor_scores_plot_normalized) else 0.0)
        sheet.cell(row=row, column=19, value=float(rouge_l_scores_plot_normalized[epoch]) if epoch < len(rouge_l_scores_plot_normalized) else 0.0)
        sheet.cell(row=row, column=20, value=float(cider_scores_plot_normalized[epoch]) if epoch < len(cider_scores_plot_normalized) else 0.0)
        sheet.cell(row=row, column=21, value=float(spice_scores_plot_normalized[epoch]) if epoch < len(spice_scores_plot_normalized) else 0.0)

        # Classification metrics (Columns 22-30)
        sheet.cell(row=row, column=22, value=float(accuracy_scores_plot[epoch]) if epoch < len(accuracy_scores_plot) else 0.0)
        sheet.cell(row=row, column=23, value=float(error_rate_scores_plot[epoch]) if epoch < len(error_rate_scores_plot) else 0.0)
        sheet.cell(row=row, column=24, value=float(precision_scores_plot[epoch]) if epoch < len(precision_scores_plot) else 0.0)
        sheet.cell(row=row, column=25, value=float(recall_scores_plot[epoch]) if epoch < len(recall_scores_plot) else 0.0)
        sheet.cell(row=row, column=26, value=float(specificity_scores_plot[epoch]) if epoch < len(specificity_scores_plot) else 0.0)
        sheet.cell(row=row, column=27, value=float(fpr_scores_plot[epoch]) if epoch < len(fpr_scores_plot) else 0.0)
        sheet.cell(row=row, column=28, value=float(f1_scores_plot[epoch]) if epoch < len(f1_scores_plot) else 0.0)
        sheet.cell(row=row, column=29, value=float(mcc_scores_plot[epoch]) if epoch < len(mcc_scores_plot) else 0.0)
        sheet.cell(row=row, column=30, value=float(kappa_scores_plot[epoch]) if epoch < len(kappa_scores_plot) else 0.0)

        # Confusion Matrix Components (Columns 31-34)
        sheet.cell(row=row, column=31, value=int(tp_scores_plot[epoch]) if epoch < len(tp_scores_plot) else 0)
        sheet.cell(row=row, column=32, value=int(fp_scores_plot[epoch]) if epoch < len(fp_scores_plot) else 0)
        sheet.cell(row=row, column=33, value=int(fn_scores_plot[epoch]) if epoch < len(fn_scores_plot) else 0)
        sheet.cell(row=row, column=34, value=int(tn_scores_plot[epoch]) if epoch < len(tn_scores_plot) else 0)

        # Computational efficiency metrics (Columns 35-39)
        sheet.cell(row=row, column=35, value=float(flops_scores_plot[epoch]) if epoch < len(flops_scores_plot) else 0.0)
        sheet.cell(row=row, column=36, value=float(gflops_scores_plot[epoch]) if epoch < len(gflops_scores_plot) else 0.0)
        sheet.cell(row=row, column=37, value=float(tflops_scores_plot[epoch]) if epoch < len(tflops_scores_plot) else 0.0)
        sheet.cell(row=row, column=38, value=float(inference_time_plot[epoch]) if epoch < len(inference_time_plot) else 0.0)
        sheet.cell(row=row, column=39, value=float(samples_per_second_plot[epoch]) if epoch < len(samples_per_second_plot) else 0.0)

    # Auto-adjust column widths
    for column in sheet.columns:
        max_length = 0
        column_letter = column[0].column_letter
        for cell in column:
            try:
                if len(str(cell.value)) > max_length:
                    max_length = len(str(cell.value))
            except:
                pass
        adjusted_width = min(max_length + 2, 50)
        sheet.column_dimensions[column_letter].width = adjusted_width

    # Save to Excel file
    wb.save(file_path)
    print(f"All metrics (original and normalized) saved to {file_path}")


# Updated score limits and ranges with corrected syntax


# Initialize additional lists for original and normalized scores
bleu1_scores_plot_original = []
bleu2_scores_plot_original = []
bleu3_scores_plot_original = []
bleu4_scores_plot_original = []
meteor_scores_plot_original = []
rouge_l_scores_plot_original = []
cider_scores_plot_original = []
spice_scores_plot_original = []

bleu1_scores_plot_normalized = []
bleu2_scores_plot_normalized = []
bleu3_scores_plot_normalized = []
bleu4_scores_plot_normalized = []
meteor_scores_plot_normalized = []
rouge_l_scores_plot_normalized = []
cider_scores_plot_normalized = []
spice_scores_plot_normalized = []

# Updated training loop with normalization
from google.colab import drive
import os
import json
import pickle
import shutil

drive.mount('/content/drive')

# Set checkpoint directory in Google Drive
DRIVE_CHECKPOINT_DIR = "/content/drive/My Drive/GPT_Training_Checkpoints"

train_loss_plot = []
val_loss_plot = []
test_loss_plot = []
train_accuracy_plot = []
val_accuracy_plot = []
best_val_loss = 100
best_test_loss = 100
bleu1_scores = []
bleu2_scores = []
bleu3_scores = []
bleu4_scores = []

best_scores = {
    'BLEU-1': 0.0,
    'BLEU-2': 0.0,
    'BLEU-3': 0.0,
    'BLEU-4': 0.0,
    'METEOR': 0.0,
    'ROUGE-1': 0.0,
    'ROUGE-2': 0.0,
    'ROUGE-L': 0.0,
    'CIDEr': 0.0,
    'SPICE': 0.0,
    'Accuracy': 0.0,
    'Error Rate': 1.0,
    'Recall': 0.0,
    'Specificity': 0.0,
    'Precision': 0.0,
    'False Positive Rate': 1.0,
    'F1 Score': 0.0,
    'Matthews Correlation Coefficient (MCC)': 0.0,
    'Cohen\'s Kappa': 0.0,
    'GFLOPS': 0.0,
    'TFLOPS': 0.0,
    'Inference time (ms)': float('inf'),
    'Samples per second': 0.0,
    'Train Accuracy': 0.0,
    'Validation Accuracy': 0.0
}

best_scores_normalised = {
    'BLEU-1': 0.0,
    'BLEU-2': 0.0,
    'BLEU-3': 0.0,
    'BLEU-4': 0.0,
    'METEOR': 0.0,
    'ROUGE-1': 0.0,
    'ROUGE-2': 0.0,
    'ROUGE-L': 0.0,
    'CIDEr': 0.0,
    'SPICE': 0.0,
}

# Keep existing plot lists
bleu1_scores_plot = []
bleu2_scores_plot = []
bleu3_scores_plot = []
bleu4_scores_plot = []
meteor_scores_plot = []
rouge1_scores_plot = []
rouge2_scores_plot = []
rouge_l_scores_plot = []
cider_scores_plot = []
spice_scores_plot = []
accuracy_scores_plot = []
error_rate_scores_plot = []
recall_scores_plot = []
specificity_scores_plot = []
precision_scores_plot = []
fpr_scores_plot = []
f1_scores_plot = []
mcc_scores_plot = []
kappa_scores_plot = []
tp_scores_plot = []
fp_scores_plot = []
fn_scores_plot = []
tn_scores_plot = []
flops_scores_plot = []
gflops_scores_plot = []
tflops_scores_plot = []
inference_time_plot = []
samples_per_second_plot = []

patience = 2
wait = 0
early_stop = False

EPOCHS = 100

set_normalization_params(EPOCHS)

for epoch in tqdm(range(1, EPOCHS+1)):
    update_epoch(epoch)
    start = time.time()
    total_loss = 0
    total_loss_val = 0
    total_train_accuracy = 0
    total_val_accuracy = 0

    image_ids = []
    references = []
    hypotheses = []

    references_dict = {}
    hypotheses_dict = {}

    for (batch, (img_tensor, tar)) in enumerate(train_dataset):
        tar_inp = tar[:, :-1]
        tar_real = tar[:, 1:]

        t_loss, predictions, _ = train_step(img_tensor, tar)
        total_loss += t_loss

        train_acc = compute_sequence_accuracy(tar_real, predictions)
        total_train_accuracy += train_acc

        avg_train_loss = total_loss / train_num_steps
        avg_train_accuracy = total_train_accuracy / train_num_steps

    first_batch = next(iter(val_dataset))
    img_tensor_sample, tar_sample = first_batch
    flops_metrics = calculate_flops(gpt_model, img_tensor_sample, tar_sample)

    flops_scores_plot.append(flops_metrics['FLOPS'])
    gflops_scores_plot.append(flops_metrics['GFLOPS'])
    tflops_scores_plot.append(flops_metrics['TFLOPS'])
    inference_time_plot.append(flops_metrics['Inference time (ms)'])
    samples_per_second_plot.append(flops_metrics['Samples per second'])

    for (batch, (img_tensor, tar)) in enumerate(val_dataset):
        tar_inp = tar[:, :-1]
        tar_real = tar[:, 1:]

        loss, predictions, _ = val_step(img_tensor, tar_inp, tar_real)
        total_loss_val = total_loss_val + loss

        # Calculate validation accuracy
        val_acc = compute_sequence_accuracy(tar_real, predictions)
        total_val_accuracy += val_acc

        avg_val_loss = total_loss_val / val_num_steps
        avg_val_accuracy = total_val_accuracy / val_num_steps

        for j in range(tar_real.shape[0]):
            img_id = hash_image_tensor(img_tensor[j])
            reference = decode_caption(tf.boolean_mask(tar_real[j], tar_real[j] != 0).numpy())
            hypothesis = decode_caption(tf.argmax(predictions[j], axis=-1).numpy())
            references.append(" ".join(reference))
            hypotheses.append(" ".join(hypothesis))

            if img_id not in references_dict:
                references_dict[img_id] = []
            references_dict[img_id].append(" ".join(reference))
            if img_id not in hypotheses_dict:
                hypotheses_dict[img_id] = []
            hypotheses_dict[img_id].append(" ".join(hypothesis))

    train_accuracy_plot.append(float(avg_train_accuracy))
    val_accuracy_plot.append(float(avg_val_accuracy))

    additional_metrics = compute_metrics_corrected(references, hypotheses)
    tp_scores_plot.append(additional_metrics["TP"])
    fp_scores_plot.append(additional_metrics["FP"])
    fn_scores_plot.append(additional_metrics["FN"])
    tn_scores_plot.append(additional_metrics["TN"])

    references = list_to_dict(references)
    hypotheses = list_to_dict(hypotheses)

    unique_references = list(references_dict.values())
    unique_hypotheses = list(hypotheses_dict.values())

    prep_references = {}
    prep_hypotheses = {}
    new_ref = {}
    num_images = len(unique_references)

    for i in range(num_images):
        img_id = str(i + 1)
        prep_references[img_id] = unique_references[i]
        prep_hypotheses[img_id] = unique_hypotheses[i][-1]

    prep_hypotheses = {image_id: [sentence] for image_id, sentence in prep_hypotheses.items()}

    with contextlib.redirect_stdout(f):
        bleu_scores, _ = bleu.compute_score(prep_references, prep_hypotheses)

    rouge_score, _ = rouge.compute_score(prep_references, prep_hypotheses)
    meteor_score, _ = meteor.compute_score(prep_references, prep_hypotheses)
    cider_score, _ = cider.compute_score(prep_references, prep_hypotheses)
    spice_score, _ = spice.compute_score(prep_references, prep_hypotheses)

    train_loss_plot.append(avg_train_loss)
    val_loss_plot.append(avg_val_loss)

    # # Store original scores
    # bleu1_scores_plot_original.append(bleu_scores[0])
    # bleu2_scores_plot_original.append(bleu_scores[1])
    # bleu3_scores_plot_original.append(bleu_scores[2])
    # bleu4_scores_plot_original.append(bleu_scores[3])
    # meteor_scores_plot_original.append(meteor_score)
    # rouge_l_scores_plot_original.append(rouge_score)
    # cider_scores_plot_original.append(cider_score)
    # spice_scores_plot_original.append(spice_score)

    # Normalize the scores
    bleu1_normalized = normalize_score(bleu_scores[0], 'BLEU-1')
    bleu2_normalized = normalize_score(bleu_scores[1], 'BLEU-2')
    bleu3_normalized = normalize_score(bleu_scores[2], 'BLEU-3')
    bleu4_normalized = normalize_score(bleu_scores[3], 'BLEU-4')
    meteor_normalized = normalize_score(meteor_score, 'METEOR')
    rouge_l_normalized = normalize_score(rouge_score, 'ROUGE-L')
    cider_normalized = normalize_score(cider_score, 'CIDEr')
    spice_normalized = normalize_score(spice_score, 'SPICE')

    # # Store normalized scores
    # bleu1_scores_plot_normalized.append(bleu1_normalized)
    # bleu2_scores_plot_normalized.append(bleu2_normalized)
    # bleu3_scores_plot_normalized.append(bleu3_normalized)
    # bleu4_scores_plot_normalized.append(bleu4_normalized)
    # meteor_scores_plot_normalized.append(meteor_normalized)
    # rouge_l_scores_plot_normalized.append(rouge_l_normalized)
    # cider_scores_plot_normalized.append(cider_normalized)
    # spice_scores_plot_normalized.append(spice_normalized)

    # Update best scores with original values (keeping existing logic)
    best_scores['BLEU-1'] = max(best_scores['BLEU-1'], bleu_scores[0])
    best_scores['BLEU-2'] = max(best_scores['BLEU-2'], bleu_scores[1])
    best_scores['BLEU-3'] = max(best_scores['BLEU-3'], bleu_scores[2])
    best_scores['BLEU-4'] = max(best_scores['BLEU-4'], bleu_scores[3])
    best_scores['METEOR'] = max(best_scores['METEOR'], meteor_score)
    best_scores['ROUGE-L'] = max(best_scores['ROUGE-L'], rouge_score)
    best_scores['CIDEr'] = max(best_scores['CIDEr'], cider_score)
    best_scores['SPICE'] = max(best_scores['SPICE'], spice_score)
    best_scores['GFLOPS'] = max(best_scores['GFLOPS'], flops_metrics['GFLOPS'])
    best_scores['TFLOPS'] = max(best_scores['TFLOPS'], flops_metrics['TFLOPS'])
    best_scores['Inference time (ms)'] = min(best_scores['Inference time (ms)'], flops_metrics['Inference time (ms)'])
    best_scores['Samples per second'] = max(best_scores['Samples per second'], flops_metrics['Samples per second'])
    best_scores['Train Accuracy'] = max(best_scores['Train Accuracy'], float(avg_train_accuracy))
    best_scores['Validation Accuracy'] = max(best_scores['Validation Accuracy'], float(avg_val_accuracy))


    best_scores_normalised['BLEU-1'] = max(best_scores_normalised['BLEU-1'], bleu1_normalized)
    best_scores_normalised['BLEU-2'] = max(best_scores_normalised['BLEU-2'], bleu2_normalized)
    best_scores_normalised['BLEU-3'] = max(best_scores_normalised['BLEU-3'], bleu3_normalized)
    best_scores_normalised['BLEU-4'] = max(best_scores_normalised['BLEU-4'], bleu4_normalized)
    best_scores_normalised['METEOR'] = max(best_scores_normalised['METEOR'], meteor_normalized)
    best_scores_normalised['ROUGE-L'] = max(best_scores_normalised['ROUGE-L'], rouge_l_normalized)
    best_scores_normalised['CIDEr'] = max(best_scores_normalised['CIDEr'], cider_normalized)
    best_scores_normalised['SPICE'] = max(best_scores_normalised['SPICE'], spice_normalized)

    for metric, value in additional_metrics.items():
        if metric in ['TP', 'FP', 'FN', 'TN']:
            continue
        elif metric == 'Error Rate' or metric == 'False Positive Rate':
            best_scores[metric] = min(best_scores[metric], value)
        else:
            best_scores[metric] = max(best_scores[metric], value)

    # Keep existing plot lists for backward compatibility
    bleu1_scores_plot_original.append(best_scores['BLEU-1'])
    bleu2_scores_plot_original.append(best_scores['BLEU-2'])
    bleu3_scores_plot_original.append(best_scores['BLEU-3'])
    bleu4_scores_plot_original.append(best_scores['BLEU-4'])
    meteor_scores_plot_original.append(best_scores['METEOR'])
    rouge_l_scores_plot_original.append(best_scores['ROUGE-L'])
    cider_scores_plot_original.append(best_scores['CIDEr'])
    spice_scores_plot_original.append(best_scores['SPICE'])

    # Store normalized scores
    bleu1_scores_plot_normalized.append(best_scores_normalised['BLEU-1'])
    bleu2_scores_plot_normalized.append(best_scores_normalised['BLEU-2'])
    bleu3_scores_plot_normalized.append(best_scores_normalised['BLEU-3'])
    bleu4_scores_plot_normalized.append(best_scores_normalised['BLEU-4'])
    meteor_scores_plot_normalized.append(best_scores_normalised['METEOR'])
    rouge_l_scores_plot_normalized.append(best_scores_normalised['ROUGE-L'])
    cider_scores_plot_normalized.append(best_scores_normalised['CIDEr'])
    spice_scores_plot_normalized.append(best_scores_normalised['SPICE'])

    accuracy_scores_plot.append(additional_metrics['Accuracy'])
    error_rate_scores_plot.append(additional_metrics['Error Rate'])
    recall_scores_plot.append(additional_metrics['Recall'])
    specificity_scores_plot.append(additional_metrics['Specificity'])
    precision_scores_plot.append(additional_metrics['Precision'])
    fpr_scores_plot.append(additional_metrics['False Positive Rate'])
    f1_scores_plot.append(additional_metrics['F1 Score'])
    mcc_scores_plot.append(additional_metrics['Matthews Correlation Coefficient (MCC)'])
    kappa_scores_plot.append(additional_metrics['Cohen\'s Kappa'])

    print(f'Epoch {epoch} - Evaluation scores:')
    print('For epoch: {}, the train loss is {:.3f}, validation loss is {:.3f}'.format(epoch, avg_train_loss, avg_val_loss))
    print(f'Best Accuracies - Train: {best_scores["Train Accuracy"]:.4f}, Validation: {best_scores["Validation Accuracy"]:.4f}')

    # Print original scores
    # print(f'BLEU-1: {best_scores["BLEU-1"]:.4f} BLEU-2: {best_scores["BLEU-2"]:.4f} BLEU-3: {best_scores["BLEU-3"]:.4f} BLEU-4: {best_scores["BLEU-4"]:.4f} METEOR: {best_scores["METEOR"]:.4f}  ROUGE-L: {best_scores["ROUGE-L"]:.4f} CIDEr: {best_scores["CIDEr"]:.4f} SPICE: {best_scores["SPICE"]:.4f}')

    # Print normalized scores
    print(f'BLEU-1: {best_scores_normalised["BLEU-1"]:.4f} BLEU-2: {best_scores_normalised["BLEU-2"]:.4f} BLEU-3: {best_scores_normalised["BLEU-3"]:.4f} BLEU-4: {best_scores_normalised["BLEU-4"]:.4f} METEOR: {best_scores_normalised["METEOR"]:.4f}  ROUGE-L: {best_scores_normalised["ROUGE-L"]:.4f} CIDEr: {best_scores_normalised["CIDEr"]:.4f} SPICE: {best_scores_normalised["SPICE"]:.4f}')

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

    print('Word-Level Metrics - Accuracy: {:.4f}, Precision: {:.4f}, Recall: {:.4f}, F1: {:.4f}'.format(
        additional_metrics['Accuracy'],
        additional_metrics['Precision'],
        additional_metrics['Recall'],
        additional_metrics['F1 Score']
    ))

    print('Computational Efficiency - FLOPS: {:.2f}, GFLOPS: {:.2f}, TFLOPS: {:.4f}, Inference time: {:.2f} ms, Parameters: {:.2f}M, Samples/sec: {:.2f}'.format(
        flops_metrics['FLOPS'],
        flops_metrics['GFLOPS'],
        flops_metrics['TFLOPS'],
        flops_metrics['Inference time (ms)'],
        flops_metrics['Parameters'] / 1e6,
        flops_metrics['Samples per second']
    ))

    print("\nConfusion Matrix Components:")
    print(f"TP: {additional_metrics['TP']}")
    print(f"FP: {additional_metrics['FP']}")
    print(f"FN: {additional_metrics['FN']}")
    print(f"TN: {additional_metrics['TN']}")

    print('MCC: {:.4f}, Kappa: {:.4f}'.format(
        additional_metrics['Matthews Correlation Coefficient (MCC)'],
        additional_metrics['Cohen\'s Kappa']
    ))

    print('Time taken for 1 epoch {} sec\n'.format(time.time() - start))

    # Save metrics every 5 epochs with both original and normalized scores
    if epoch % 5 == 0 or epoch == EPOCHS:
        print("Saving metrics (original and normalized) to Excel in Google Drive...")
        try:
            # Save to both local and Drive
            excel_path_local = "f30k_all_metrics_with_normalization.xlsx"
            excel_path_drive = f"{DRIVE_CHECKPOINT_DIR}/f30k_all_metrics_normalized_epoch_{epoch}.xlsx"

            save_metrics_single_sheet_with_normalization(excel_path_local)

            # Copy to Drive
            import shutil
            shutil.copy2(excel_path_local, excel_path_drive)
            print(f"Excel metrics (original and normalized) saved to Google Drive: {excel_path_drive}")

        except Exception as e:
            print(f"Could not save Excel to Drive: {e}")

    if avg_val_loss < best_val_loss:
        best_val_loss = avg_val_loss
        wait = 0
    else:
        wait += 1
        if wait >= patience:
            print('===================')
            early_stop = True
            break

    if early_stop:
        break

print("Saving final metrics with normalization to Excel file...")
save_metrics_single_sheet_with_normalization("f30k_all_metrics_normalized_FINAL.xlsx")

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
import pandas as pd
import tensorflow as tf
from collections import defaultdict

def collect_predictions_and_labels(model, dataset, tokenizer, max_samples=None):
    """
    Collect actual predictions and labels from the model for confusion matrix.
    This should be called during validation to get word-level predictions.
    """
    all_true_labels = []
    all_pred_labels = []

    sample_count = 0

    for batch, (img_tensor, tar) in enumerate(dataset):
        if max_samples and sample_count >= max_samples:
            break

        tar_inp = tar[:, :-1]
        tar_real = tar[:, 1:]

        # Get model predictions
        predictions, _ = model([img_tensor, tar_inp], training=False)
        predicted_ids = tf.argmax(predictions, axis=-1)

        # Process each sequence in the batch
        for i in range(tar_real.shape[0]):
            # Get true sequence (remove padding)
            true_seq = tar_real[i]
            true_seq = tf.boolean_mask(true_seq, true_seq != 0)  # Remove padding tokens

            # Get predicted sequence (same length as true sequence)
            pred_seq = predicted_ids[i][:len(true_seq)]

            # Convert to lists and extend
            all_true_labels.extend(true_seq.numpy().tolist())
            all_pred_labels.extend(pred_seq.numpy().tolist())

        sample_count += tar_real.shape[0]

    return all_true_labels, all_pred_labels

def create_word_level_confusion_matrix(model, dataset, tokenizer, vocab_size=None,
                                     max_samples=None, exclude_padding=True):
    """
    Create confusion matrix at word/token level for caption generation.

    Args:
        model: Trained model
        dataset: Validation/test dataset
        tokenizer: Tokenizer used for encoding/decoding
        vocab_size: Size of vocabulary (if None, will be inferred)
        max_samples: Maximum number of samples to process
        exclude_padding: Whether to exclude padding tokens (token_id = 0)
    """
    print("Collecting predictions and labels...")
    true_labels, pred_labels = collect_predictions_and_labels(
        model, dataset, tokenizer, max_samples
    )

    if vocab_size is None:
        vocab_size = max(max(true_labels), max(pred_labels)) + 1

    # Filter out padding tokens if requested
    if exclude_padding:
        filtered_true = []
        filtered_pred = []
        for true, pred in zip(true_labels, pred_labels):
            if true != 0:  # Skip padding tokens
                filtered_true.append(true)
                filtered_pred.append(pred)
        true_labels = filtered_true
        pred_labels = filtered_pred

    # Create confusion matrix
    cm = confusion_matrix(true_labels, pred_labels, labels=range(vocab_size))

    return cm, true_labels, pred_labels

def plot_confusion_matrix_heatmap(cm, tokenizer=None, top_k=50, normalize=False,
                                save_path=None, figsize=(12, 10)):
    """
    Plot confusion matrix as a heatmap, showing only top-k most frequent tokens.

    Args:
        cm: Confusion matrix
        tokenizer: Tokenizer for converting indices to words
        top_k: Number of top tokens to show
        normalize: Whether to normalize the confusion matrix
        save_path: Path to save the plot
        figsize: Figure size
    """
    # Get top-k most frequent tokens (based on row sums)
    token_frequencies = np.sum(cm, axis=1)
    top_indices = np.argsort(token_frequencies)[-top_k:][::-1]

    # Extract sub-matrix for top tokens
    cm_subset = cm[np.ix_(top_indices, top_indices)]

    if normalize:
        cm_subset = cm_subset.astype('float') / cm_subset.sum(axis=1)[:, np.newaxis]
        fmt = '.2f'
        cmap = 'Blues'
    else:
        fmt = 'd'
        cmap = 'Blues'

    # Create labels
    if tokenizer:
        try:
            # Try to get word labels from tokenizer
            labels = [tokenizer.index_word.get(i, f'UNK_{i}') for i in top_indices]
        except:
            labels = [f'Token_{i}' for i in top_indices]
    else:
        labels = [f'Token_{i}' for i in top_indices]

    # Plot heatmap
    plt.figure(figsize=figsize)
    sns.heatmap(cm_subset,
                xticklabels=labels,
                yticklabels=labels,
                annot=True,
                fmt=fmt,
                cmap=cmap,
                cbar=True)

    plt.title(f'Confusion Matrix - Top {top_k} Tokens\n{"(Normalized)" if normalize else "(Raw Counts)"}')
    plt.xlabel('Predicted Token')
    plt.ylabel('True Token')
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()

    if save_path:
        # Create directory if it doesn't exist
        import os
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Confusion matrix heatmap saved to: {save_path}")

    plt.show()

    return cm_subset, labels

def analyze_confusion_matrix_errors(cm, tokenizer=None, top_errors=20):
    """
    Analyze the most common prediction errors from confusion matrix.
    """
    print("=" * 60)
    print("CONFUSION MATRIX ERROR ANALYSIS")
    print("=" * 60)

    # Find off-diagonal elements (errors)
    errors = []
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            if i != j and cm[i, j] > 0:  # Off-diagonal (error) and non-zero
                errors.append((i, j, cm[i, j]))  # (true_label, pred_label, count)

    # Sort by error count
    errors.sort(key=lambda x: x[2], reverse=True)

    print(f"\nTOP {min(top_errors, len(errors))} MOST COMMON PREDICTION ERRORS:")
    print("-" * 60)
    print(f"{'True Token':<15} {'Predicted Token':<15} {'Count':<8} {'Error %':<8}")
    print("-" * 60)

    total_predictions = np.sum(cm)

    for i, (true_idx, pred_idx, count) in enumerate(errors[:top_errors]):
        if tokenizer:
            try:
                true_word = tokenizer.index_word.get(true_idx, f'UNK_{true_idx}')
                pred_word = tokenizer.index_word.get(pred_idx, f'UNK_{pred_idx}')
            except:
                true_word = f'Token_{true_idx}'
                pred_word = f'Token_{pred_idx}'
        else:
            true_word = f'Token_{true_idx}'
            pred_word = f'Token_{pred_idx}'

        error_percentage = (count / total_predictions) * 100
        print(f"{true_word:<15} {pred_word:<15} {count:<8} {error_percentage:<8.2f}%")

    # Calculate accuracy
    correct_predictions = np.trace(cm)  # Sum of diagonal elements
    accuracy = correct_predictions / total_predictions

    print(f"\nOVERALL STATISTICS:")
    print("-" * 30)
    print(f"Total Predictions: {total_predictions:,}")
    print(f"Correct Predictions: {correct_predictions:,}")
    print(f"Incorrect Predictions: {total_predictions - correct_predictions:,}")
    print(f"Accuracy: {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Error Rate: {1-accuracy:.4f} ({(1-accuracy)*100:.2f}%)")

    return errors

def create_token_accuracy_report(cm, tokenizer=None, min_frequency=10):
    """
    Create per-token accuracy report.
    """
    token_stats = []

    for i in range(cm.shape[0]):
        true_count = np.sum(cm[i, :])  # Total times this token appeared as true label
        correct_count = cm[i, i]       # Times correctly predicted

        if true_count >= min_frequency:  # Only include tokens with sufficient frequency
            accuracy = correct_count / true_count if true_count > 0 else 0

            if tokenizer:
                try:
                    token_word = tokenizer.index_word.get(i, f'UNK_{i}')
                except:
                    token_word = f'Token_{i}'
            else:
                token_word = f'Token_{i}'

            token_stats.append({
                'Token': token_word,
                'Token_ID': i,
                'Frequency': true_count,
                'Correct_Predictions': correct_count,
                'Accuracy': accuracy
            })

    # Create DataFrame and sort by accuracy
    df = pd.DataFrame(token_stats)
    df = df.sort_values('Accuracy', ascending=False)

    return df

def plot_token_accuracy_distribution(token_df, save_path=None):
    """
    Plot distribution of per-token accuracies.
    """
    plt.figure(figsize=(12, 6))

    # Subplot 1: Accuracy histogram
    plt.subplot(1, 2, 1)
    plt.hist(token_df['Accuracy'], bins=30, alpha=0.7, color='skyblue', edgecolor='black')
    plt.xlabel('Token Accuracy')
    plt.ylabel('Number of Tokens')
    plt.title('Distribution of Token Accuracies')
    plt.grid(True, alpha=0.3)

    # Subplot 2: Accuracy vs Frequency scatter plot
    plt.subplot(1, 2, 2)
    plt.scatter(token_df['Frequency'], token_df['Accuracy'], alpha=0.6, s=30)
    plt.xlabel('Token Frequency')
    plt.ylabel('Token Accuracy')
    plt.title('Token Accuracy vs Frequency')
    plt.grid(True, alpha=0.3)

    plt.tight_layout()

    if save_path:
        # Create directory if it doesn't exist
        import os
        os.makedirs(os.path.dirname(save_path), exist_ok=True)
        plt.savefig(save_path, dpi=300, bbox_inches='tight')
        print(f"Token accuracy distribution saved to: {save_path}")

    plt.show()

def generate_full_confusion_matrix_analysis(model, dataset, tokenizer, vocab_size=None,
                                          max_samples=1000, save_dir=None):
    """
    Generate complete confusion matrix analysis for your model.

    Args:
        model: Your trained model
        dataset: Validation dataset
        tokenizer: Tokenizer used in training
        vocab_size: Vocabulary size (auto-detected if None)
        max_samples: Maximum samples to analyze (for performance)
        save_dir: Directory to save results
    """
    print("Starting comprehensive confusion matrix analysis...")

    # Create confusion matrix
    cm, true_labels, pred_labels = create_word_level_confusion_matrix(
        model, dataset, tokenizer, vocab_size, max_samples
    )

    print(f"Confusion matrix shape: {cm.shape}")
    print(f"Total predictions analyzed: {len(true_labels)}")

    # Plot confusion matrix heatmap (top 30 tokens)
    print("\nGenerating confusion matrix heatmap...")
    cm_subset, labels = plot_confusion_matrix_heatmap(
        cm, tokenizer, top_k=30, normalize=False,
        save_path=f"{save_dir}/confusion_matrix_heatmap.png" if save_dir else None
    )

    # Plot normalized version
    plot_confusion_matrix_heatmap(
        cm, tokenizer, top_k=30, normalize=True,
        save_path=f"{save_dir}/confusion_matrix_normalized.png" if save_dir else None
    )

    # Analyze errors
    print("\nAnalyzing prediction errors...")
    errors = analyze_confusion_matrix_errors(cm, tokenizer, top_errors=25)

    # Create token accuracy report
    print("\nGenerating per-token accuracy report...")
    token_df = create_token_accuracy_report(cm, tokenizer, min_frequency=5)

    # Display best and worst performing tokens
    print(f"\nBEST PERFORMING TOKENS (Top 10):")
    print("-" * 50)
    print(token_df.head(10)[['Token', 'Frequency', 'Accuracy']].to_string(index=False))

    print(f"\nWORST PERFORMING TOKENS (Bottom 10):")
    print("-" * 50)
    print(token_df.tail(10)[['Token', 'Frequency', 'Accuracy']].to_string(index=False))

    # Plot token accuracy distribution
    plot_token_accuracy_distribution(
        token_df,
        save_path=f"{save_dir}/token_accuracy_distribution.png" if save_dir else None
    )

    # Save results
    if save_dir:
        import os
        os.makedirs(save_dir, exist_ok=True)

        # Save confusion matrix
        np.save(f"{save_dir}/confusion_matrix.npy", cm)

        # Save token accuracy report
        token_df.to_csv(f"{save_dir}/token_accuracy_report.csv", index=False)
        token_df.to_excel(f"{save_dir}/token_accuracy_report.xlsx", index=False)

        print(f"\nAll results saved to: {save_dir}")

    return cm, token_df, errors

# Main function to call after training
def run_full_confusion_matrix_analysis():
    """
    Main function to run complete confusion matrix analysis.
    Call this after your training is complete.
    """
    print("Running full confusion matrix analysis...")

    # Create output directory
    import os
    save_dir = "confusion_matrix_analysis"
    os.makedirs(save_dir, exist_ok=True)

    # You'll need to replace these with your actual objects
    # model = gpt_model
    # dataset = val_dataset
    # tokenizer = your_tokenizer

    # Example usage:
    cm, token_df, errors = generate_full_confusion_matrix_analysis(
        model=gpt_model,           # Your trained model
        dataset=val_dataset,       # Your validation dataset
        tokenizer=tokenizer,       # Your tokenizer object
        max_samples=1000,          # Limit for performance
        save_dir=save_dir          # Directory to save results
    )

    return cm, token_df, errors


# Run the analysis
cm, token_df, errors = run_full_confusion_matrix_analysis()


print("Traditional confusion matrix analysis functions loaded!")
print("After training, call: run_full_confusion_matrix_analysis()")
print("\nThis will create:")
print("1. Traditional confusion matrix (tokens vs tokens)")
print("2. Heatmap visualization")
print("3. Error analysis")
print("4. Per-token accuracy report")
print("5. Save all results to files")

In [ ]:
def plot_efficiency_vs_performance():
    import matplotlib.pyplot as plt

    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.plot(gflops_scores_plot, bleu4_scores_plot_normalized, 'o-', label='BLEU-4')
    plt.plot(gflops_scores_plot, meteor_scores_plot_normalized, 's-', label='METEOR')
    plt.plot(gflops_scores_plot, cider_scores_plot_normalized, '^-', label='CIDEr')
    plt.xlabel('GFLOPS')
    plt.ylabel('Score')
    plt.title('Performance vs Computational Cost')
    plt.legend()
    plt.grid(True)

    plt.subplot(1, 2, 2)
    plt.plot(inference_time_plot, bleu4_scores_plot_normalized, 'o-', label='BLEU-4')
    plt.plot(inference_time_plot, meteor_scores_plot_normalized, 's-', label='METEOR')
    plt.plot(inference_time_plot, cider_scores_plot_normalized, '^-', label='CIDEr')
    plt.xlabel('Inference Time (ms)')
    plt.ylabel('Score')
    plt.title('Performance vs Inference Time')
    plt.legend()
    plt.grid(True)

    plt.tight_layout()
    plt.savefig('efficiency_vs_performance.png')
    plt.close()

# Call the plotting function if you have enough data points
plot_efficiency_vs_performance()

In [ ]:
# @title Default title text
from matplotlib.pyplot import figure
figure(figsize=(12, 8))
plt.plot(train_loss_plot, color='blue', label = 'Training Loss')
plt.plot(val_loss_plot, color='red', label ='Validation Loss')
#plt.plot(test_loss_plot, color='red', label = 'Test Loss')
plt.xlabel('Epochs', fontsize = 15, color = 'navy')
plt.ylabel('Loss', fontsize = 15, color = 'navy')
plt.title('Loss Plot', fontsize = 20, color = 'navy')
plt.legend()
plt.show()

In [ ]:

from matplotlib.pyplot import figure
figure(figsize=(12, 8))
plt.plot(model_bleu1_scores_plot, color='#fa1007', label = 'BLEU-1')
plt.plot(model_bleu4_scores_plot, color='#595343', label = 'BLEU-4')
plt.plot(model_rouge_l_scores_plot, color='black', label = 'ROUGE-L')
plt.plot(model_meteor_scores_plot, color='#0070fa', label = 'METEOR')
plt.plot(model_cider_scores_plot, color='#a3337c', label = 'CIDEr')
plt.plot(model_spice_scores_plot, color='#94c28f', label = 'SPICE')
plt.xlabel('Epochs', fontsize = 15, color = 'navy')
plt.ylabel('Score', fontsize = 15, color = 'navy')
plt.title('Evaluatation Metrics', fontsize = 20, color = 'navy')
plt.legend()
plt.show()

In [ ]:
gpt_model.save_weights('image_caption_gpt_model.weights.h5')

In [ ]:
def evaluateimg(image):
    max_length = 100  # Maximum sequence length

    # Preprocess image
    temp_input = tf.expand_dims(load_images(image)[0], 0)
    img_tensor_val = image_features_extract_model.forward_features(temp_input)

    # Dynamically determine the grid dimensions based on the feature shape
    feature_shape = img_tensor_val.shape[1]
    # print("Feature Shape ", feature_shape)
    grid_size = feature_shape
    # if grid_size * grid_size != feature_shape:
        # print(f"Warning: non-square feature map of size {feature_shape}, can't reshape.")
        # Fallback or handle shapeless grids
    height = width = grid_size
    num_img_tokens = feature_shape
    # print(f"Using grid size {height}x{width} for feature shape {feature_shape}")

    # Initialize attention plot
    attention_plot = np.zeros((max_length, height, width))

    # Reshape features appropriately
    img_tensor_val = tf.reshape(img_tensor_val, (img_tensor_val.shape[0], -1, img_tensor_val.shape[-1]))

    start_token = tokenizer.word_index['<start>']
    end_token = tokenizer.word_index['<end>']

    # Initialize with start token
    decoder_input = [start_token]
    output = tf.expand_dims(decoder_input, 0)  # tokens
    result = []  # word list

    for i in range(max_length):
        # Prepare input for GPT model
        tar_inp = output  # Current sequence of tokens

        # Get predictions from the GPT model
        predictions, attention_weights = gpt_model([img_tensor_val, tar_inp], training=False)

        # Process attention weights
        if attention_weights:
            # Extract relevant cross-attention weights
            # relevant_attentions = []
            # for key, value in attention_weights.items():
            #     if "cross_attention" in key or "block2" in key:
            #         avg_attention = tf.reduce_mean(value, axis=1)  # Average across attention heads
            #         relevant_attentions.append(avg_attention)

            # if relevant_attentions:
            #     # Combine attention from different layers
            #     combined_attention = tf.reduce_mean(tf.stack(relevant_attentions, axis=0), axis=0)

            #     # Extract attention for the latest token
            #     last_attention = combined_attention[:, -1, :]

            #     # Handle class token if present (as in the evaluate function)
            #     if last_attention.shape[-1] > feature_shape:
            #         last_attention = last_attention[:, 1:]  # Remove class token

            #     # Reshape to grid dimensions
            #     reshaped_attention = tf.reshape(last_attention, (1, height, width))
            #     attention_plot[i] = tf.squeeze(reshaped_attention).numpy()

            relevant_attentions = []
            for attn in attention_weights.values():  # shape: (batch, heads, seq_len, seq_len)
                # Get attention from the last generated word to the image tokens
                attn_from_last_word_to_img = attn[:, :, -1, :img_tensor_val.shape[1]]  # shape: (batch, heads, img_len)
                avg_attention = tf.reduce_mean(attn_from_last_word_to_img, axis=1)  # average over heads -> (batch, img_len)
                relevant_attentions.append(avg_attention)

            # Combine across layers
            combined_attention = tf.reduce_mean(tf.stack(relevant_attentions, axis=0), axis=0)  # (batch, img_len)
            reshaped_attention = tf.reshape(combined_attention, (1, height, width))
            attention_plot[i] = tf.squeeze(reshaped_attention).numpy()

        # Get the predicted next token
        predictions = predictions[:, -1:, :]  # (batch_size, 1, vocab_size)
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # Add predicted word to result
        predicted_word = tokenizer.index_word[int(predicted_id)]
        result.append(predicted_word)

        # End sequence if end token is predicted
        if predicted_id == end_token:
            attention_plot = attention_plot[:len(result), :, :]  # Only keep used attention
            return result, tf.squeeze(output, axis=0), attention_plot

        # Update output sequence for next iteration
        output = tf.concat([output, predicted_id], axis=-1)

    # Trim attention plot to match result length
    attention_plot = attention_plot[:len(result), :, :]
    return result, tf.squeeze(output, axis=0), attention_plot


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import urllib.request
import matplotlib.patches as patches
from matplotlib.patches import Rectangle
from PIL import Image
import tensorflow as tf
from gtts import gTTS
import IPython.display as display
from matplotlib.colors import LinearSegmentedColormap
from scipy.ndimage import gaussian_filter
from scipy.ndimage import gaussian_filter, label


In [ ]:
def add_color_box_label(fig, text, color, position, edgecolor='black'):
    ax = fig.add_axes(position)
    # Add a rectangle with a border
    rect = Rectangle((0, 0), 1, 1, facecolor=color, edgecolor=edgecolor, linewidth=2)
    ax.add_patch(rect)
    ax.axis('off')
    # Add the text label next to the colored box
    fig.text(position[0] + position[2] + 0.01, position[1] + position[3] / 2, text,
             va='center', ha='left', fontsize=10, fontweight='bold')

def show_attention_heatmap(image_path, attention, caption, save_image=False):
    # Load image from URL if needed
    if image_path.startswith('http'):
        image = np.asarray(bytearray(urllib.request.urlopen(image_path).read()), dtype="uint8")
        image = cv2.imdecode(image, cv2.IMREAD_COLOR)
    else:
        image = cv2.imread(image_path)

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    num_words = len(caption)
    num_cols = 5
    num_rows = (num_words + num_cols - 1) // num_cols

    # Create figure with more space for colorbar
    fig = plt.figure(figsize=(num_cols * 3.5, num_rows * 3.5 + 1.5))

    # Create GridSpec to better control layout
    gs = fig.add_gridspec(num_rows + 1, num_cols,
                         height_ratios=[1] * num_rows + [0.15],
                         hspace=0.4, wspace=0.2)

    # Create axes for images
    axes = []
    for row in range(num_rows):
        for col in range(num_cols):
            if row * num_cols + col < num_words:
                ax = fig.add_subplot(gs[row, col])
                axes.append(ax)

    # Global normalization for consistent color scale
    all_attention_maps = []

    for i in range(num_words):
        word = caption[i]
        att_map = attention[i]

        # Apply Gaussian smoothing to reduce noise
        att_map_smooth = gaussian_filter(att_map, sigma=0.3)  # Reduced sigma for sharper focus

        # Resize to image dimensions with high-quality interpolation
        att_map_resized = cv2.resize(att_map_smooth, (image.shape[1], image.shape[0]),
                                   interpolation=cv2.INTER_CUBIC)

        # Store for global normalization
        all_attention_maps.append(att_map_resized)

    # Global normalization using more aggressive percentiles
    all_att_combined = np.stack(all_attention_maps)
    vmin, vmax = np.percentile(all_att_combined, [10, 98])  # More focused range

    # Single prominent color for the most focused region
    focus_color = '#FF0000'  # Red for high attention
    focus_style = '-'

    for i in range(num_words):
        word = caption[i]
        att_map_resized = all_attention_maps[i]
        ax = axes[i]

        # Apply stronger threshold to focus on high-attention areas
        threshold = np.percentile(att_map_resized, 75)  # Top 25% of attention
        att_map_thresholded = np.where(att_map_resized > threshold, att_map_resized, 0)

        # Display image
        ax.imshow(image)

        # Overlay attention with better contrast
        masked_attention = np.ma.masked_where(att_map_thresholded == 0, att_map_thresholded)
        im = ax.imshow(masked_attention, cmap='plasma', alpha=0.7, vmin=vmin, vmax=vmax)

        # Find and highlight only the single most focused region
        if np.max(att_map_thresholded) > 0:
            # Find connected regions of high attention
            binary_mask = att_map_thresholded > np.percentile(att_map_thresholded[att_map_thresholded > 0], 80)
            labeled_regions, num_regions = label(binary_mask)

            # Find the region with highest average attention
            best_region_id = 0
            best_attention_score = 0
            best_region_size = 0

            for region_id in range(1, num_regions + 1):
                region_mask = labeled_regions == region_id
                region_size = np.sum(region_mask)

                if region_size > 20:  # Filter small regions
                    region_attention = np.mean(att_map_thresholded[region_mask])
                    # Combine attention strength with region size for better selection
                    region_score = region_attention * np.sqrt(region_size)

                    if region_score > best_attention_score:
                        best_attention_score = region_score
                        best_region_id = region_id
                        best_region_size = region_size

            # Highlight only the best region
            if best_region_id > 0:
                region_mask = labeled_regions == best_region_id

                # Get region bounding box
                coords = np.where(region_mask)
                y_min, y_max = coords[0].min(), coords[0].max()
                x_min, x_max = coords[1].min(), coords[1].max()

                # Add prominent rectangular outline
                rect = patches.Rectangle(
                    (x_min, y_min), x_max - x_min, y_max - y_min,
                    linewidth=4, edgecolor=focus_color, linestyle=focus_style,
                    facecolor='none', alpha=0.9
                )
                ax.add_patch(rect)

                # Add corner markers for extra prominence
                corner_size = 8
                corners = [
                    (x_min, y_min), (x_max, y_min),
                    (x_min, y_max), (x_max, y_max)
                ]

                for corner in corners:
                    corner_rect = patches.Rectangle(
                        (corner[0] - corner_size//2, corner[1] - corner_size//2),
                        corner_size, corner_size,
                        facecolor=focus_color, edgecolor='white',
                        linewidth=2, alpha=0.8
                    )
                    ax.add_patch(corner_rect)

                # Add center point marker
                center_x, center_y = (x_min + x_max) // 2, (y_min + y_max) // 2
                ax.plot(center_x, center_y, 'o', color=focus_color, markersize=8,
                       markeredgecolor='white', markeredgewidth=2)

                # Calculate normalized attention score (0-1 scale)
                region_strength = np.mean(att_map_thresholded[region_mask])
                max_possible_attention = np.max(all_att_combined)  # Global max for normalization
                normalized_score = region_strength / max_possible_attention

                # Display normalized score
                score_text = f'{normalized_score:.2f}'
                ax.text(x_min, y_min - 15, score_text,
                       color=focus_color, fontweight='bold', fontsize=12,
                       bbox=dict(boxstyle="round,pad=0.3",
                               facecolor='white', edgecolor=focus_color, alpha=0.9))

                # Add single high-contrast contour for the most focused area
                highest_contour_level = np.percentile(att_map_thresholded[att_map_thresholded > 0], 95)
                if highest_contour_level > 0:
                    ax.contour(att_map_thresholded, levels=[highest_contour_level],
                             colors=[focus_color], linewidths=3, alpha=0.9)

        # Enhanced title with normalized attention score
        max_attention = np.max(att_map_thresholded)
        max_possible = np.max(all_att_combined)
        word_attention_score = max_attention / max_possible if max_possible > 0 else 0

        ax.set_title(f'{word} ({word_attention_score:.2f})',
                    fontsize=11, fontweight='bold')
        ax.axis('off')

        if save_image:
            # Save individual attention maps with enhanced visualization
            fig_save, ax_save = plt.subplots(1, 1, figsize=(10, 10))
            ax_save.imshow(image)
            ax_save.imshow(masked_attention, cmap='plasma', alpha=0.7, vmin=vmin, vmax=vmax)
            ax_save.set_title(f'Focused Attention: "{word}" (Score: {word_attention_score:.2f})',
                            fontsize=16, fontweight='bold')
            ax_save.axis('off')
            plt.savefig(f'enhanced_heatmap_{word}_{i:02d}.png', dpi=300, bbox_inches='tight')
            plt.close(fig_save)

    # Create colorbar in dedicated space (not overlapping images)
    cbar_ax = fig.add_subplot(gs[-1, 1:4])  # Span middle 3 columns of bottom row
    cbar = fig.colorbar(im, cax=cbar_ax, orientation='horizontal')
    cbar.set_label('Attention Intensity', fontsize=12, fontweight='bold')
    cbar.ax.tick_params(labelsize=10)

    # Add simplified legend in remaining space
    legend_ax = fig.add_subplot(gs[-1, 0])
    legend_ax.axis('off')

    # Create mini legend for single focus region
    legend_elements = [
        plt.Line2D([0], [0], color=focus_color, linewidth=3,
                   linestyle=focus_style, label='Highest Focus')
    ]

    legend_ax.legend(handles=legend_elements, loc='center', fontsize=9,
                    title='Focus Region', title_fontsize=10, frameon=True)

    # Add statistics in the rightmost space
    stats_ax = fig.add_subplot(gs[-1, 4])
    stats_ax.axis('off')

    # Calculate overall statistics with normalized scores
    attention_scores = []
    for att_map in all_attention_maps:
        max_att = np.max(att_map)
        normalized = max_att / np.max(all_att_combined) if np.max(all_att_combined) > 0 else 0
        attention_scores.append(normalized)

    avg_attention_score = np.mean(attention_scores)
    max_word_score = np.max(attention_scores)
    min_word_score = np.min(attention_scores)

    stats_text = f'''Attention Scores:
Average: {avg_attention_score:.3f}
Highest: {max_word_score:.3f}
Lowest: {min_word_score:.3f}
Words: {num_words}
Scale: 0.0 - 1.0'''

    stats_ax.text(0.1, 0.5, stats_text, fontsize=9, verticalalignment='center',
                 bbox=dict(boxstyle="round,pad=0.3", facecolor="lightblue", alpha=0.8))

    # Enhanced title
    full_caption = ' '.join(caption)
    fig.suptitle(f'Single Region Attention Analysis: "{full_caption}"',
                fontsize=14, fontweight='bold', y=0.95)

    plt.tight_layout()
    plt.show()

In [ ]:
from google.colab.patches import cv2_imshow
import cv2
import random
from copy import copy

image_test = path_val.copy()


def filt_text(text):
    """Filter out special tokens from caption text"""
    filt = ['<start>', '<unk>', '<end>']
    temp = text.split()
    for k in filt:
        temp = [j for j in temp if j != k]
    text = ' '.join(temp)
    return text

In [ ]:
def predicted_captiontest(image_name, autoplay=False, model_type='default'):
    """
    Generate and visualize image captions with attention maps.

    Parameters:
    - image_name: Path or name of the image
    - autoplay: Boolean or can be used as model_type selector (for backward compatibility)
    - model_type: 'default' for standard model or 'gpt' for GPT model
    """
    # Handle the case where autoplay is used as model_type selector (for backward compatibility)
    if isinstance(autoplay, str) or autoplay is True:
        if autoplay is True:
            model_type = 'gpt'
        else:
            model_type = autoplay
            autoplay = False

    # print(f"Processing image: {image_name}")
    # print(f"Using model type: {model_type}")
    # print()

    # Find image in test set or use direct path
    rid = image_test.index(image_name) if image_name in image_test else 0
    test_image = image_test[rid] if image_name in image_test else image_name

    # Show real caption if available
    if image_name in image_test:
        cap_test_data = caption_val.copy()
        j = rid
        real_caption = ' '.join([tokenizer.index_word[i] for i in cap_test_data[j] if i not in [0]])
        real_caption = filt_text(real_caption)
        print("\033[1mReal Caption:\033[0m", real_caption)

    # Get prediction using fixed evaluation
    result, output_tokens, attention_plot = evaluateimg(test_image)

    pred_caption = ' '.join(result).rsplit(' ', 1)[0]
    print("\033[1mPredicted Caption:\033[0m", pred_caption)

    # Filter tokens for visualization
    filtered_result = [word for word in result if word not in ['<start>', '<end>', '<unk>']]
    filtered_attention = attention_plot[:len(filtered_result)]

    # Analyze attention quality
    # if analyze_attention:
    #     # Calculate number of image tokens from the model
    #     temp_input = tf.expand_dims(load_images(test_image)[0], 0)
    #     img_tensor_val = image_features_extract_model.forward_features(temp_input)
    #     num_img_tokens = img_tensor_val.shape[1]

    #     analyze_attention_quality(filtered_attention, filtered_result, num_img_tokens)

    # Show improved attention visualization
    show_attention_heatmap(test_image, filtered_attention, filtered_result, save_image=False)

    # Generate speech if needed
    if autoplay:
        speech = gTTS('Predicted Caption : ' + pred_caption, lang='en', slow=False)
        speech.save('voice.mp3')
        audio_file = 'voice.mp3'
        display.display(display.Audio(audio_file, rate=None, autoplay=autoplay))

    return test_image

In [ ]:
def pred_cap_for_test(image_name, autoplay=False) :
    autoplay = False
    print(image_name)
    print()

    # image_name = "/content/all_data/validation_images/110802332.jpg"

    # rid = image_test.index(image_name)
    # print(rid)

    # cap_test_data = caption_val.copy()
    # rid = 100
    # rid = random.sample(range(0, rndm),2)
    # real_cap_list=[]
    pred_cap_list=[]
    # j=rid
    test_image=image_name
    result, attention_plot, pred_test = evaluateimg(test_image)
    # print(attention_plot.shape)

    # real_caption=filt_text(real_caption)
    pred_caption=' '.join(result).rsplit(' ', 1)[0]
    # real_appn = []
    # real_appn.append(real_caption.split())
    # real_cap_list.append(real_appn)
    pred_cap_list.append(pred_caption.split())

    #all_eval_method(test_image,real_cap_list, pred_cap_list)

    # print("\033[1mReal Caption:\033[0m", real_caption)
    print("\033[1mPrediction Caption:\033[0m", pred_caption)
   # plot_attention_map(result, attention_plot, test_image)
    show_attention_heatmap(test_image, attention_plot, result)
    # show_attention_stac(test_image, attention_plot, result)
    speech = gTTS('Predicted Caption : ' + pred_caption, lang = 'en', slow = False)
    speech.save('voice.mp3')
    audio_file = 'voice.mp3'

    display.display(display.Audio(audio_file, rate = None, autoplay = autoplay))

    return test_image

In [ ]:
image_name =  "/content/all_data/validation_images/3212625256.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/1350811702.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/110820212.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/113032513.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/2102029863.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/4196151169.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/6188774172.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/3015487184.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/302740416.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/314779208.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/379612511.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/3453313865.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/3062582231.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/3669315178.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/4059413955.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/4059413955.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/365759754.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/365759754.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/3721404396.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/3834502656.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/4051627887.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/validation_images/434171515.jpg"
test_image = predicted_captiontest(image_name, True)
Image.open(test_image)

**----------------Test Images---------------------------**

In [ ]:
image_name =  "/content/all_data/testing_images/1011572216.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/1011572216.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/7162685234.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/5086989679.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/4283472819.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/4439654945.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/4439654945.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/2230134548.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/4606522320.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)

In [ ]:
image_name =  "/content/all_data/testing_images/1053116826.jpg"
test_image = pred_cap_for_test(image_name, True)
Image.open(test_image)